In [439]:
# DESeq2 comparison of DE genes between samples - Nov2016

In [440]:
# Packages

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn3
from varname import nameof

In [441]:
def countDEGs(df):
    try:
        lfc = df["log2FoldChange"]  
    except KeyError:
        lfc = df["log2FoldChange_x"]  
    lfc = lfc.tolist()
    neg_count = len(list(filter(lambda x: (x < 0), lfc)))
    pos_count = len(list(filter(lambda x: (x >= 0), lfc)))
    print("positive :",pos_count)
    print("negative :",neg_count)

def extractCalicoblasticGenes(dfList):
    genesListTotal = []
    for i in dfList:
        try:
            genesList = i['genes_x'].tolist()
        except:
            genesList = i['genes'].tolist()
        genesListTotal = genesListTotal + genesList
    genesListTotal = list(set(genesListTotal))
    return genesListTotal 

def candidateListGenerator(all_annot,dfList,outputname):
    genesList = extractCalicoblasticGenes(dfList)
    all_annot_trimmed = all_annot.loc[all_annot['genes'].isin(genesList)].reset_index(drop=True)
    pfam_annot = all_annot_trimmed['pfam_annotation']
    all_annot_trimmed = all_annot_trimmed[['genes']]
    all_annot_trimmed['pfam_annotation'] = np.array(pfam_annot)
    all_annot_trimmed = all_annot_trimmed.drop_duplicates(subset='genes',keep='first', inplace=False, ignore_index=True)
    all_annot_trimmed.to_csv('/Users/mmeynadier/Documents/Astroides/comparative_transcriptomics_astroides/data/net/8_functionalAnnotation/calicoblasticGenes/'+outputname+'.csv',index=False,header=True)

In [442]:
def getRBHdata(spList):
    sp1 = spList[0] ; sp2 = spList[1]
    forward = pd.read_csv('~/Documents/PhD/species/crossSpecies/Astroides_Stylophora/RBH/'+sp1+'_'+sp2+'.txt',sep="\t")
    backward = pd.read_csv('~/Documents/PhD/species/crossSpecies/Astroides_Stylophora/RBH/'+sp2+'_'+sp1+'.txt',sep="\t") 
    backward[['qseqid','sseqid']] = backward[['sseqid','qseqid']] 
    RBHdf = pd.merge(forward,backward,on=['qseqid','sseqid'],how='inner')
    astNames = RBHdf['sseqid'].tolist()
    trimAstNames = []
    for i in astNames:
        trim = i.split('|')[1]
        trimAstNames.append(trim)
    RBHdf['sseqid'] = trimAstNames
    RBHdf = RBHdf.rename(columns={"qseqid":"genes"})
    return RBHdf

orthologs = getRBHdata(['Astroides','Stylophora'])
orthologs

,genes,sseqid,pident_x,qcovs_x,qlen_x,slen_x,length_x,bitscore_x,evalue_x,pident_y,qcovs_y,qlen_y,slen_y,length_y,bitscore_y,evalue_y
0,Spis10031_1,STRG.43792,61.069,75,173,145,131,145.0,2.410000e-45,58.741,98,145,173,143,157.0,5.710000e-50
1,Spis10033_1,STRG.43799,56.140,81,118,138,114,115.0,2.240000e-34,56.140,83,138,118,114,115.0,2.960000e-34
2,Spis10075_1,STRG.40523,66.346,30,344,110,104,153.0,2.280000e-46,66.346,95,110,344,104,153.0,8.240000e-47
3,Spis10110_1,STRG.25791,22.791,70,227,419,215,48.9,7.940000e-07,22.791,51,419,227,215,48.9,1.660000e-06
4,Spis10164_1,STRG.42271,92.361,16,903,144,144,276.0,3.920000e-88,92.361,100,144,903,144,276.0,1.010000e-88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13440,Spis_XP_022810653_1,STRG.17617,60.947,100,166,188,169,185.0,2.050000e-60,60.947,90,188,166,169,185.0,2.480000e-60
13441,Spis_XP_022810654_1,STRG.18786,81.406,99,2316,2285,2318,3848.0,0.000000e+00,81.363,99,2285,2316,2318,3821.0,0.000000e+00
13442,Spis_XP_022810657_1,STRG.15108,74.336,82,137,309,113,171.0,1.180000e-53,68.182,42,309,137,132,174.0,1.900000e-54
13443,Spis_XP_022810659_1,STRG.26455,63.860,100,482,472,487,605.0,0.000000e+00,63.860,99,472,482,487,598.0,0.000000e+00


In [443]:
# Filter by gene enrichment

def filterGO(dataset,df,dfString):
    with open("/Users/mmeynadier/Documents/Astroides/comparative_transcriptomics_astroides/data/net/8_functionalAnnotation/ontologizer/associationFile.ids") as f:
        l = f.readlines()
    associationDict = {}
    for i in l:
        gene = i.split("\t")[0]
        go = i.split("\t")[1]
        go = go.split("\n")[0]
        go = go.split(",")
        associationDict[gene] = go
    path = ("/Users/mmeynadier/Documents/Astroides/comparative_transcriptomics_astroides/data/net/8_functionalAnnotation/ontologizer/results/"+dataset+"/")   
    dir_list = os.listdir(path)
    for i in dir_list:
        if dfString in i:
            with open(path+i) as f:
                dfGO = f.readlines()
    goDict = {}
    for i in range(len(dfGO)):
        if i == 0:
            dfGO[i] = dfGO[i].split("\n")[0]
            colName = dfGO[i].split("\t")
            for j in colName:
                goDict[j] = []
        else:
            row = dfGO[i].split("\n")[0]
            row = row.split("\t")
            for j in range(len(row)):
                goDict[colName[j]].append(row[j])
    geneList = []


#filterGO("nov2016",sa_VS_gm,"sa_VS_gm")

In [444]:
def trimAnnotEggnog(path):
    curedFile = []
    with open(path) as f:
        contents = f.readlines()
    f.close()
    for i in contents:
        if 'STRG' in i:
            curedFile.append(i)
    geneName = [] ; function = [] ; protName = [] ; pfamDomain = []
    for i in curedFile:
        splitElement = i.split('\t')
        tx = splitElement[0]
        tx = tx.rsplit('_',1)[0]
        tx = tx.split('|',1)[1]
        geneName.append(tx)
        function.append(splitElement[7])
        protName.append(splitElement[8])
        pfamDomain.append(splitElement[-1].replace('\n',''))
    dic = {'genes':geneName,'function':function,'protein':protName,'pfam_domains':pfamDomain}
    df=pd.DataFrame(dic)
    df=df.drop_duplicates(subset='genes',ignore_index=False)
    return df

A_eggnog = trimAnnotEggnog('/Users/mmeynadier/Documents/Astroides/comparative_transcriptomics_astroides/data/net/8_functionalAnnotation/eggnogAnnotAstroides')
A_eggnog

,genes,function,protein,pfam_domains
0,STRG.10001,alpha-L-fucosidase activity,FUCA1,"Alpha_L_fucos,Fucosidase_C"
1,STRG.10002,RNA binding,-,"La,RRM_1"
2,STRG.10003,"WD domain, G-beta repeat",-,"F-box-like,WD40"
3,STRG.10004,Glycine rich protein,LCN1,Lipocalin
4,STRG.10006,Matrixin,MMP28,"Hemopexin,PG_binding_1,Peptidase_M10"
...,...,...,...,...
23605,STRG.9987,metalloaminopeptidase activity,NPEPPS,"ERAP1_C,Peptidase_M1"
23606,STRG.9988,metalloaminopeptidase activity,NPEPPS,"ERAP1_C,Peptidase_M1"
23607,STRG.9989,collagen,COL4A1,"C4,Collagen"
23608,STRG.9996,zinc ion binding,-,-


In [445]:
def trimAnnotPfam(path):
    curedFile = []
    with open(path) as f:
        contents = f.readlines()
    for i in contents:
        if 'STRG' in i:
            curedFile.append(i)
    geneNames=[] ; pfamAnnot= [] ; pfamCode = []
    separator = "PF"
    for i in range(len(curedFile)):
        gene = curedFile[i].split(" - ",1)[0]
        gene = gene.replace(".p1","")
        gene = gene.rsplit(".",1)[0]
        geneNames.append(gene)
        splitDash = curedFile[i].split(" - ",1)[1]
        splitAnnot = splitDash.split(separator,1)[0] ; splitAnnot = splitAnnot.strip()  
        splitCode = splitDash.split(separator,1)[1] ; splitCode = splitCode.split(".",1)[0] ; splitCode = separator + splitCode
        pfamAnnot.append(splitAnnot) ; pfamCode.append(splitCode)
    pfamAnnotCured = []
    for i in pfamAnnot:
        try:
            annot = i.split(" ")[1]
        except IndexError:
            annot = ""
        pfamAnnotCured.append(annot)
        
    dic={'genes':geneNames,'pfam_annotation':pfamAnnotCured,'pfam_code':pfamCode}
    mergeDf=pd.DataFrame(dic) 
    mergeDf = mergeDf.drop_duplicates(subset='genes',ignore_index=False)
    return mergeDf

A_pfam = trimAnnotPfam('/Users/mmeynadier/Documents/Astroides/comparative_transcriptomics_astroides/data/net/8_functionalAnnotation/ast_all_domtbl.out') 
A_pfam

,genes,pfam_annotation,pfam_code
0,STRG.26188,1-cysPrx_C,PF10417
1,STRG.8355,1-cysPrx_C,PF10417
3,STRG.23285,1-cysPrx_C,PF10417
4,STRG.21056,14-3-3,PF00244
5,STRG.41137,14-3-3,PF00244
...,...,...,...
114138,STRG.39853,Zw10,PF06248
114141,STRG.35796,ZZ,PF00569
114142,STRG.35800,ZZ,PF00569
114166,STRG.16010,ZZ,PF00569


In [446]:
A_all_annot = pd.merge(A_pfam,A_eggnog,on='genes',how='inner')
A_all_annot

,genes,pfam_annotation,pfam_code,function,protein,pfam_domains
0,STRG.26188,1-cysPrx_C,PF10417,negative regulation of male germ cell prolifer...,PRDX4,"1-cysPrx_C,AhpC-TSA"
1,STRG.8355,1-cysPrx_C,PF10417,Peroxiredoxin,PRDX1,"1-cysPrx_C,AhpC-TSA"
2,STRG.23285,1-cysPrx_C,PF10417,peroxiredoxin activity,PRDX6,"1-cysPrx_C,AhpC-TSA"
3,STRG.21056,14-3-3,PF00244,Belongs to the 14-3-3 family,YWHAE,14-3-3
4,STRG.41137,14-3-3,PF00244,Belongs to the 14-3-3 family,14-3-3zeta,14-3-3
...,...,...,...,...,...,...
19138,STRG.29108,ZU5,PF00791,P53-induced death domain protein 1,PIDD1,"Death,LRR_8,Peptidase_S68,ZU5"
19139,STRG.39853,Zw10,PF06248,membrane addition at site of cytokinesis,ZW10,Zw10
19140,STRG.35796,ZZ,PF00569,calcium ion binding,ZZEF1,"ANAPC10,ZZ"
19141,STRG.35800,ZZ,PF00569,calcium ion binding,ZZEF1,"ANAPC10,ZZ"


In [447]:
# Trim calicoblast cell expression of Stylophora

stGeneAnnot = pd.read_csv('/Users/mmeynadier/Documents/Astroides/comparative_transcriptomics_astroides/data/net/8_functionalAnnotation/Spis_gene_annotation.tsv',sep='\t')
stGeneAnnot = stGeneAnnot.rename(columns={"Spis10006_1":"genes"})

stCellTypeFC = pd.read_csv('/Users/mmeynadier/Documents/Astroides/comparative_transcriptomics_astroides/data/net/8_functionalAnnotation/Spis_adult_cell_type_gene_FC.tsv',sep='\t')
stcalicoblast = stCellTypeFC.loc[stCellTypeFC['calicoblast'] > 2]
stcalicoblast.index.name = 'genes'

stMerge = pd.merge(stGeneAnnot,stcalicoblast,on='genes',how='inner')
stMerge = stMerge.filter(['genes','CUB/'])
stMerge = stMerge.dropna()

annot = list(stMerge['CUB/'])
cureAnnot = []
for i in annot:
    a = i.split('/')[0]
    cureAnnot.append(a)
stMerge['CUB/'] = cureAnnot
stMerge = stMerge.rename(columns={"CUB/":"pfam_annotation"})
stMerge

,genes,pfam_annotation
1,Spis14986_1,Spond_N
7,Spis_XP_022778944_1,MORN
8,Spis_XP_022779261_1,bZIP_Maf
9,Spis_XP_022779543_1,Trypsin
10,Spis_XP_022779802_1,Lipase
...,...,...
149,Spis_XP_022807783_1,CH
150,Spis_XP_022807817_1,F5_F8_type_C
152,Spis_XP_022809492_1,Lectin_C
153,Spis_XP_022810032_1,I-set


In [448]:
stcalicoblast

,alga-hosting_cells,calicoblast,cnidocyte,digestive_filaments,epidermis_1,epidermis_2,gastrodermis,gastrodermis_musclelike,germline_sperm,gland_1,...,neuron_14,neuron_2,neuron_3,neuron_4,neuron_5,neuron_6,neuron_7,neuron_8,neuron_9,unknown
genes,,,,,,,,,,,,,,,,,,,,,
Spis11609_1,0.903613,2.313918,1.016917,1.050643,1.052563,1.100606,0.974076,1.021021,0.812151,1.316163,...,1.085429,1.097457,0.812151,0.938753,0.884021,1.036374,1.164475,0.812151,0.812151,1.068413
Spis14986_1,1.038890,2.433970,1.000000,1.052302,1.073700,1.522699,1.034664,1.000000,1.000000,1.000000,...,1.167346,1.000000,1.199393,1.022131,1.029415,1.276086,1.000000,1.101509,1.000000,1.000000
Spis15711_1,0.979386,2.039369,0.857901,1.012717,1.202398,1.057957,1.283077,0.912571,0.857901,1.314960,...,1.001467,0.857901,0.857901,1.041513,0.933819,1.094755,1.152021,0.857901,0.991613,1.128599
Spis15868_1,1.012819,2.801197,1.122649,1.021882,2.242104,1.293944,0.984539,1.069020,0.896593,0.994674,...,1.046634,1.053413,1.122530,0.976210,0.975935,0.896593,0.896593,0.896593,1.036335,1.648328
Spis16543_1,1.528036,2.343112,0.858320,1.077543,1.710868,2.297349,1.710528,1.023387,0.858320,1.237758,...,1.680495,1.008445,1.138486,0.934538,0.883567,0.858320,1.043255,0.858320,0.858320,0.966304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
orphan_peak_5406,1.000000,2.727878,1.228264,1.059399,1.049441,1.088665,1.041922,0.980941,0.980941,0.980941,...,0.980941,0.980941,1.353120,1.068049,1.009795,0.980941,0.980941,0.980941,0.980941,1.104353
orphan_peak_5879,1.006473,2.491461,1.000000,1.069762,1.019756,1.000000,1.019081,1.000000,1.000000,1.109394,...,1.000000,1.351298,1.000000,1.022131,1.046660,1.000000,1.215462,1.000000,1.155860,1.000000
orphan_peak_6856,1.921218,2.210104,1.066809,1.761010,1.125051,1.736935,1.704010,1.877515,0.604193,0.803833,...,0.910785,1.363042,0.956626,0.956009,0.842312,1.667207,1.000000,0.665524,0.945650,0.910207


In [449]:
# merge calicoblast and RBH

calicoOrthologs = pd.merge(stcalicoblast,orthologs,on="genes",how="inner")
astroCalicoOrthologs = calicoOrthologs['sseqid']
astroCalicoOrthologs = astroCalicoOrthologs.rename("genes")
astroCalicoOrthologsAnnot = pd.merge(astroCalicoOrthologs,A_all_annot,how="inner",on="genes")

In [450]:
print(astroCalicoOrthologs.to_list())

['STRG.39230', 'STRG.3949', 'STRG.15962', 'STRG.6653', 'STRG.43502', 'STRG.5191', 'STRG.43285', 'STRG.6501', 'STRG.15221', 'STRG.17809', 'STRG.24608', 'STRG.2562', 'STRG.22516', 'STRG.36707', 'STRG.40202', 'STRG.11602', 'STRG.36750', 'STRG.25673', 'STRG.33907', 'STRG.684', 'STRG.38830', 'STRG.9141', 'STRG.17657', 'STRG.24218', 'STRG.9381', 'STRG.10010', 'STRG.43534', 'STRG.4561', 'STRG.40596', 'STRG.10865', 'STRG.2388', 'STRG.30319', 'STRG.30115', 'STRG.36335', 'STRG.17404', 'STRG.30411', 'STRG.23422', 'STRG.27038', 'STRG.7429', 'STRG.15706', 'STRG.205', 'STRG.22761', 'STRG.22760', 'STRG.10554', 'STRG.2771', 'STRG.17863', 'STRG.14048', 'STRG.2130', 'STRG.752', 'STRG.30826', 'STRG.27319', 'STRG.1630', 'STRG.42770', 'STRG.44199', 'STRG.22232', 'STRG.4892', 'STRG.39182', 'STRG.19949', 'STRG.36913', 'STRG.36909', 'STRG.16194', 'STRG.16167', 'STRG.14787', 'STRG.7699', 'STRG.677', 'STRG.22634', 'STRG.38089', 'STRG.27366', 'STRG.9250', 'STRG.9248', 'STRG.14831', 'STRG.1093', 'STRG.15060', 'ST

In [451]:
rPath = os.getcwd()
candidatesGenesPath = rPath + "/../R_scripts/candidateGenes.csv"
dataPath = rPath + "/../../data/net/7_deseq2/annotatedGenome/adult/DESeq2_results_adult_preliminarySamples_"
pv_VS_gm_path = dataPath+"pv_VS_gm.csv"
sa_VS_gm_path = dataPath+"sa_VS_gm.csv"
pv_VS_sa_path = dataPath+"pv_VS_sa.csv"

In [452]:
pv_VS_gm = pd.read_csv(pv_VS_gm_path,sep=',')
sa_VS_gm = pd.read_csv(sa_VS_gm_path,sep=',')
pv_VS_sa = pd.read_csv(pv_VS_sa_path,sep=',')

In [453]:
pv_VS_gm = pv_VS_gm.rename(columns={'Unnamed: 0':'genes'})
sa_VS_gm = sa_VS_gm.rename(columns={'Unnamed: 0':'genes'})
pv_VS_sa = pv_VS_sa.rename(columns={'Unnamed: 0':'genes'})

In [454]:
pv_VS_gm

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,STRG.7522,306.989738,-6.925785,0.647078,-10.703163,9.836188e-27,1.921893e-22
1,STRG.9273,173.046337,3.396245,0.352600,9.632006,5.857445e-22,5.722431e-18
2,STRG.34730,3631.087049,2.002629,0.224498,8.920456,4.643727e-19,3.024459e-15
3,STRG.19570,928.165470,3.816884,0.457763,8.338130,7.547426e-17,3.686729e-13
4,STRG.12557,56.481430,2.716353,0.341183,7.961579,1.698577e-15,5.531416e-12
...,...,...,...,...,...,...,...
23318,STRG.9964,1.527015,1.581335,1.668124,0.947972,3.431437e-01,NaN
23319,STRG.9972,2.367394,1.410959,2.522712,0.559303,5.759552e-01,NaN
23320,STRG.9977,1.414399,1.522631,2.914014,0.522520,6.013082e-01,NaN
23321,STRG.9983,2.368578,-2.541510,1.650237,-1.540087,1.235391e-01,NaN


In [455]:
pv_VS_gm_ortho = pd.merge(pv_VS_gm,astroCalicoOrthologsAnnot,on="genes",how="inner")
pv_VS_gm_ortho

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code,function,protein,pfam_domains
0,STRG.2728,187.894380,2.028750,0.447548,4.533036,0.000006,0.000684,EGF,PF00008,neurogenic locus notch homolog protein,-,EGF
1,STRG.14579,574.905907,-0.430089,0.164024,-2.622111,0.008739,0.147321,ADAM_CR_2,PF17771,metalloendopeptidase activity,ADAMTS10,"ADAM_spacer1,PLAC,Pep_M12B_propep,Reprolysin,T..."
2,STRG.9381,6.671076,-1.897785,0.749036,-2.533636,0.011289,0.171698,Cadherin,PF00028,homophilic cell adhesion via plasma membrane a...,-,Cadherin
3,STRG.22233,54.494841,1.195304,0.488944,2.444662,0.014499,0.199924,EF-hand_5,PF13202,calcium ion binding,RCN3,"EF-hand_5,EF-hand_8"
4,STRG.35172,85.659917,0.720447,0.301417,2.390199,0.016839,0.217607,Mito_carr,PF00153,Belongs to the mitochondrial carrier (TC 2.A.2...,SLC25A1,Mito_carr
...,...,...,...,...,...,...,...,...,...,...,...,...
69,STRG.22761,258.969521,-0.002319,0.215573,-0.010759,0.991416,0.997662,PDZ,PF00595,macromolecule localization,-,"PDZ,PH"
70,STRG.12083,4.242650,1.172778,0.959510,1.222267,0.221607,NaN,VWA,PF00092,calcium ion transport,-,"VGCC_alpha2,VWA,VWA_2,VWA_N"
71,STRG.2388,3.496191,3.597436,2.054516,1.750990,0.079948,NaN,Cytochrom_B561,PF03188,Ferric-chelate reductase,FRRS1,"Cytochrom_B561,DOMON,Reeler"
72,STRG.36750,4.494780,0.901942,0.948578,0.950836,0.341687,NaN,VWA,PF00092,negative regulation of growth rate,-,"Ank_2,CUB,EGF,EGF_CA,Ephrin_rec_like,FXa_inhib..."


In [456]:
sa_VS_gm_ortho = pd.merge(sa_VS_gm,astroCalicoOrthologsAnnot,on="genes",how="inner")
sa_VS_gm_ortho

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code,function,protein,pfam_domains
0,STRG.36913,372.759356,-1.289245,0.225717,-5.711788,1.117954e-08,0.000001,Thyroglobulin_1,PF00086,Trypsin-like serine protease,-,Trypsin
1,STRG.2728,187.894380,2.231496,0.447371,4.988024,6.099992e-07,0.000044,EGF,PF00008,neurogenic locus notch homolog protein,-,EGF
2,STRG.22760,376.284540,-0.831534,0.192264,-4.324967,1.525547e-05,0.000669,I-set,PF07679,myosin light chain kinase activity,SPEG,"I-set,Pkinase,SPEG_u2,fn3"
3,STRG.7429,860.328030,-0.727074,0.171172,-4.247632,2.160418e-05,0.000887,PDZ,PF00595,negative regulation of secondary metabolite bi...,RAPGEF2,"PDZ,RA,RasGEF,RasGEF_N,cNMP_binding"
4,STRG.30826,549.362188,-0.423542,0.109221,-3.877854,1.053819e-04,0.003312,SH2,PF00017,B-cell linker,BLNK,SH2
...,...,...,...,...,...,...,...,...,...,...,...,...
69,STRG.5191,18.401229,-0.036843,0.760571,-0.048441,9.613651e-01,0.990096,p450,PF00067,heme binding,TBXAS1,p450
70,STRG.22761,258.969521,0.009540,0.215968,0.044173,9.647665e-01,0.990510,PDZ,PF00595,macromolecule localization,-,"PDZ,PH"
71,STRG.11602,1365.535251,-0.007187,0.210577,-0.034131,9.727723e-01,0.993582,CH,PF00307,actin filament network formation,Fim,"CH,EF-hand_5,EF-hand_6,EF-hand_7"
72,STRG.2388,3.496191,1.067277,2.140433,0.498627,6.180425e-01,NaN,Cytochrom_B561,PF03188,Ferric-chelate reductase,FRRS1,"Cytochrom_B561,DOMON,Reeler"


In [457]:
pv_VS_sa_ortho = pd.merge(pv_VS_sa,astroCalicoOrthologsAnnot,on="genes",how="inner")
pv_VS_sa_ortho

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code,function,protein,pfam_domains
0,STRG.36913,372.759356,1.267661,0.240253,5.276363,1.317731e-07,0.000036,Thyroglobulin_1,PF00086,Trypsin-like serine protease,-,Trypsin
1,STRG.27038,20.454941,6.071868,1.300310,4.669555,3.018530e-06,0.000392,Forkhead,PF00250,sequence-specific DNA binding,-,Forkhead
2,STRG.22760,376.284540,0.911819,0.204337,4.462322,8.107611e-06,0.000786,I-set,PF07679,myosin light chain kinase activity,SPEG,"I-set,Pkinase,SPEG_u2,fn3"
3,STRG.24608,47.640255,2.278279,0.536574,4.245973,2.176469e-05,0.001611,PX,PF00787,podosome assembly,NCF1,"NECFESHC,PX,SH3_1,p47_phox_C"
4,STRG.25673,16569.083914,1.384478,0.368366,3.758437,1.709784e-04,0.006608,fn3,PF00041,fibroblast growth factor-activated receptor ac...,-,Pkinase_Tyr
...,...,...,...,...,...,...,...,...,...,...,...,...
69,STRG.17404,560.712712,-0.006657,0.219110,-0.030381,9.757632e-01,0.991445,fn3,PF00041,fibroblast growth factor-activated receptor ac...,-,Pkinase_Tyr
70,STRG.12083,4.242650,0.260525,0.965957,0.269707,7.873858e-01,NaN,VWA,PF00092,calcium ion transport,-,"VGCC_alpha2,VWA,VWA_2,VWA_N"
71,STRG.2388,3.496191,2.530159,2.099340,1.205216,2.281198e-01,NaN,Cytochrom_B561,PF03188,Ferric-chelate reductase,FRRS1,"Cytochrom_B561,DOMON,Reeler"
72,STRG.36750,4.494780,4.624031,1.506306,3.069781,2.142155e-03,NaN,VWA,PF00092,negative regulation of growth rate,-,"Ank_2,CUB,EGF,EGF_CA,Ephrin_rec_like,FXa_inhib..."


In [ ]:
candidateListGenerator(A_all_annot,[pv_VS_sa_ortho],'nov2016_ortho')

In [458]:
# Filter by p-adjusted 

pv_VS_gm = pv_VS_gm[pv_VS_gm['padj']<0.05]
pv_VS_gm = pv_VS_gm[(pv_VS_gm['log2FoldChange']<-2) | (pv_VS_gm['log2FoldChange']>2)]
sa_VS_gm = sa_VS_gm[sa_VS_gm['padj']<0.05]
sa_VS_gm = sa_VS_gm[(sa_VS_gm['log2FoldChange']<-2) | (sa_VS_gm['log2FoldChange']>2)]
pv_VS_sa = pv_VS_sa[pv_VS_sa['padj']<0.05]
pv_VS_sa = pv_VS_sa[(pv_VS_sa['log2FoldChange']<-2) | (pv_VS_sa['log2FoldChange']>2)]

In [459]:
pv_VS_gm_ortho2 = pd.merge(pv_VS_gm,astroCalicoOrthologs,on="genes",how="inner")
pv_VS_gm_ortho2

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,STRG.2728,187.89438,2.02875,0.447548,4.533036,0.000006,0.000684


In [460]:
sa_VS_gm_ortho2 = pd.merge(sa_VS_gm,astroCalicoOrthologs,on="genes",how="inner")
sa_VS_gm_ortho2

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,STRG.2728,187.894380,2.231496,0.447371,4.988024,6.099992e-07,0.000044
1,STRG.27038,20.454941,-4.795770,1.275916,-3.758689,1.708058e-04,0.004941
2,STRG.9381,6.671076,-3.342687,0.953016,-3.507482,4.523696e-04,0.010651


In [461]:
pv_VS_sa_ortho2 = pd.merge(pv_VS_sa,astroCalicoOrthologs,on="genes",how="inner")
pv_VS_sa_ortho2

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,STRG.27038,20.454941,6.071868,1.300310,4.669555,0.000003,0.000392
1,STRG.24608,47.640255,2.278279,0.536574,4.245973,0.000022,0.001611
2,STRG.10010,52.927430,2.573937,0.852588,3.018968,0.002536,0.039035


In [462]:
# Get common genes

common_gm = pd.merge(sa_VS_gm,pv_VS_gm,how="inner",on="genes")
common_pv = pd.merge(pv_VS_sa,pv_VS_gm,how="inner",on="genes")
common_sa = pd.merge(pv_VS_sa,sa_VS_gm,how="inner",on="genes")
common_all = pd.merge(common_gm,pv_VS_sa,how="inner",on="genes")

In [463]:
common_all_annotate = pd.merge(common_all,A_all_annot,on="genes")
common_all_annotate

,genes,baseMean_x,log2FoldChange_x,lfcSE_x,stat_x,pvalue_x,padj_x,baseMean_y,log2FoldChange_y,lfcSE_y,...,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code,function,protein,pfam_domains
0,STRG.11739,38.563759,7.249396,1.025393,7.069870,1.550793e-12,7.045532e-10,38.563759,3.500712,1.056719,...,-3.748684,0.683711,-5.482853,4.185219e-08,0.000017,2OG-FeII_Oxy,PF03171,oxidoreductase activity,-,"2OG-FeII_Oxy,DIOX_N"
1,STRG.3688,43.524535,-4.217813,0.646274,-6.526358,6.738823e-11,1.531433e-08,43.524535,-2.180879,0.556314,...,2.036934,0.694207,2.934187,3.344228e-03,0.046641,Dicer_dimer,PF03368,conversion of ds siRNA to ss siRNA involved in...,DICER1,"DEAD,Dicer_dimer,Helicase_C,PAZ,ResIII,Ribonuc..."
2,STRG.1212,44.566699,-2.381232,0.742070,-3.208906,1.332409e-03,2.473060e-02,44.566699,-5.964554,0.958353,...,-3.583322,1.010947,-3.544521,3.933273e-04,0.011093,BACK,PF07707,Leucine Rich repeat,-,LRR_6


In [464]:
common_pv_annot = pd.merge(common_pv,A_all_annot,how='inner',on='genes')
common_pv_annot = common_pv_annot.sort_values(by=['log2FoldChange_x'],ascending=False)

In [465]:
common_sa_annot = pd.merge(common_sa,A_all_annot,how='inner',on='genes')
common_sa_annot = common_sa_annot.sort_values(by=['log2FoldChange_x'],ascending=False)

In [466]:
common_gm_annot = pd.merge(common_gm,A_all_annot,how='inner',on='genes')
common_gm_annot = common_gm_annot.sort_values(by=['log2FoldChange_x'],ascending=False)

In [467]:
pv_sa_annot = pd.merge(pv_VS_sa,A_all_annot,how='inner',on='genes')

In [468]:
pv_sa_ortho = pd.merge(pv_VS_sa,astroCalicoOrthologs,how='inner',on='genes')

In [469]:
pv_sa_ortho

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,STRG.27038,20.454941,6.071868,1.300310,4.669555,0.000003,0.000392
1,STRG.24608,47.640255,2.278279,0.536574,4.245973,0.000022,0.001611
2,STRG.10010,52.927430,2.573937,0.852588,3.018968,0.002536,0.039035


In [470]:
pv_gm_annot = pd.merge(pv_VS_gm,A_all_annot,how='inner',on='genes')
pv_gm_annot

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code,function,protein,pfam_domains
0,STRG.7522,306.989738,-6.925785,0.647078,-10.703163,9.836188e-27,1.921893e-22,Ribosomal_S13_N,PF08069,small ribosomal subunit rRNA binding,RPS13,"Ribosomal_S13_N,Ribosomal_S15"
1,STRG.34730,3631.087049,2.002629,0.224498,8.920456,4.643727e-19,3.024459e-15,F5_F8_type_C,PF00754,Common central domain of tyrosinase,-,Tyrosinase
2,STRG.19570,928.165470,3.816884,0.457763,8.338130,7.547426e-17,3.686729e-13,Lectin_C,PF00059,carbohydrate binding,-,Lectin_C
3,STRG.12557,56.481430,2.716353,0.341183,7.961579,1.698577e-15,5.531416e-12,Laminin_G_3,PF13385,endocytic hemoglobin import,-,CFEM
4,STRG.1815,173.248787,-9.128663,1.289899,-7.077037,1.472695e-12,2.877498e-09,PUB,PF09409,Spermatogenesis-associated protein 2,SPATA2,-
...,...,...,...,...,...,...,...,...,...,...,...,...
159,STRG.30962,12.514205,5.951431,1.895527,3.139723,1.691076e-03,4.943598e-02,Mei4,PF13971,Meiosis-specific protein MEI4-like,MEI4,Mei4
160,STRG.7010,10.843497,2.497061,0.795613,3.138538,1.697930e-03,4.943598e-02,CLN3,PF02487,lysosomal lumen pH elevation,BTN1,CLN3
161,STRG.11343,24.885520,2.363259,0.753357,3.136971,1.707028e-03,4.948607e-02,7tm_1,PF00001,Belongs to the G-protein coupled receptor 1 fa...,-,7tm_1
162,STRG.34707,18.588591,2.104179,0.671144,3.135211,1.717305e-03,4.963671e-02,G_glu_transpept,PF01019,glutathione hydrolase activity,-,G_glu_transpept


In [471]:
pv_gm_ortho= pd.merge(pv_VS_gm,astroCalicoOrthologsAnnot,how='inner',on='genes')
pv_gm_ortho

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code,function,protein,pfam_domains
0,STRG.2728,187.89438,2.02875,0.447548,4.533036,0.000006,0.000684,EGF,PF00008,neurogenic locus notch homolog protein,-,EGF


In [472]:
sa_gm_annot = pd.merge(sa_VS_gm,A_all_annot,how='inner',on='genes')

In [473]:
sa_gm_ortho= pd.merge(sa_VS_gm,astroCalicoOrthologsAnnot,how='inner',on='genes')
sa_gm_ortho

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code,function,protein,pfam_domains
0,STRG.2728,187.894380,2.231496,0.447371,4.988024,6.099992e-07,0.000044,EGF,PF00008,neurogenic locus notch homolog protein,-,EGF
1,STRG.27038,20.454941,-4.795770,1.275916,-3.758689,1.708058e-04,0.004941,Forkhead,PF00250,sequence-specific DNA binding,-,Forkhead
2,STRG.9381,6.671076,-3.342687,0.953016,-3.507482,4.523696e-04,0.010651,Cadherin,PF00028,homophilic cell adhesion via plasma membrane a...,-,Cadherin


In [474]:
sa_VS_gm

,genes,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
0,STRG.22543,1117.035941,3.002539,0.269627,11.135878,8.391374e-29,1.677436e-24
1,STRG.9251,1002.929040,2.713298,0.265280,10.228036,1.485007e-24,1.484265e-20
2,STRG.2596,920.824289,3.232219,0.332291,9.727067,2.311627e-22,1.540314e-18
3,STRG.26789,713.504324,3.756289,0.393063,9.556450,1.218650e-21,6.090204e-18
4,STRG.22007,2604.536160,2.945216,0.314713,9.358420,8.093774e-21,3.235891e-17
...,...,...,...,...,...,...,...
1342,STRG.34999,11.345895,-2.488925,0.844921,-2.945747,3.221756e-03,4.795671e-02
1344,STRG.5143,9.591165,-2.775247,0.942838,-2.943503,3.245208e-03,4.823175e-02
1345,STRG.17514,7.303393,-3.131270,1.063966,-2.943017,3.250309e-03,4.827168e-02
1352,STRG.13412,6.685193,2.672630,0.910858,2.934189,3.344202e-03,4.940917e-02


In [475]:
exclusive_pv_annot = pd.merge(common_pv_annot,sa_gm_annot,how='left',on='genes')
exclusive_pv_annot = exclusive_pv_annot.sort_values(by=['log2FoldChange_x'],ascending=False)
print(exclusive_pv_annot.to_string())

         genes   baseMean_x  log2FoldChange_x   lfcSE_x     stat_x      pvalue_x        padj_x   baseMean_y  log2FoldChange_y   lfcSE_y     stat_y      pvalue_y        padj_y pfam_annotation_x pfam_code_x                                                       function_x protein_x                                          pfam_domains_x   baseMean  log2FoldChange     lfcSE      stat        pvalue          padj pfam_annotation_y pfam_code_y                                                       function_y protein_y                                         pfam_domains_y
0   STRG.34941    13.303101          6.046081  1.648507   3.667611  2.448271e-04  8.244240e-03    13.303101          5.088195  1.376154   3.697402  2.178176e-04  1.159656e-02           DUF1647     PF07801                            Protein of unknown function (DUF1647)         -                                                 DUF1647        NaN             NaN       NaN       NaN           NaN           NaN               NaN 

In [476]:
exclusive_sa_annot = pd.merge(common_sa_annot,pv_gm_annot,how='left',on='genes')
exclusive_sa_annot = exclusive_sa_annot.sort_values(by=['log2FoldChange_x'],ascending=False)
print(exclusive_sa_annot.to_string())

         genes    baseMean_x  log2FoldChange_x   lfcSE_x    stat_x      pvalue_x        padj_x    baseMean_y  log2FoldChange_y   lfcSE_y    stat_y      pvalue_y        padj_y pfam_annotation_x pfam_code_x                                                              function_x protein_x                                                 pfam_domains_x   baseMean  log2FoldChange     lfcSE      stat        pvalue          padj pfam_annotation_y pfam_code_y                                                       function_y protein_y                                         pfam_domains_y
0   STRG.19457     24.845327          8.166715  2.043215  3.996993  6.415213e-05  3.430971e-03     24.845327         -7.027266  1.959396 -3.586445  3.352173e-04  8.460850e-03               fn3     PF00041                                         Sidekick cell adhesion molecule     PTPRQ                                         Ig_3,Y_phosphatase,fn3        NaN             NaN       NaN       NaN           NaN     

In [477]:
exclusive_gm_annot = pd.merge(common_gm_annot,pv_sa_annot,how='left',on='genes')
exclusive_gm_annot = exclusive_gm_annot.sort_values(by=['log2FoldChange_x'],ascending=False)
print(exclusive_gm_annot.to_string())

         genes   baseMean_x  log2FoldChange_x   lfcSE_x    stat_x      pvalue_x        padj_x   baseMean_y  log2FoldChange_y   lfcSE_y    stat_y      pvalue_y        padj_y pfam_annotation_x                pfam_code_x                                                                                          function_x protein_x                                                                                      pfam_domains_x   baseMean  log2FoldChange     lfcSE      stat        pvalue      padj pfam_annotation_y pfam_code_y                                                       function_y protein_y                                         pfam_domains_y
0    STRG.1286    27.308161         20.475908  3.633454  5.635384  1.746689e-08  1.867183e-06    27.308161         21.280768  3.632133  5.859029  4.655827e-09  2.393953e-06            Chromo                    PF00385                                                                               Integrase core domain         -              

In [478]:
sa_gm_cali = pd.merge(sa_gm_annot,stMerge,how='inner',on='pfam_annotation')
sa_gm_cali =  sa_gm_cali.drop_duplicates(subset='genes_x',ignore_index=True)
pv_gm_cali = pd.merge(pv_gm_annot,stMerge,how='inner',on='pfam_annotation')
pv_gm_cali =  pv_gm_cali.drop_duplicates(subset='genes_x',ignore_index=True)
pv_sa_cali= pd.merge(pv_sa_annot,stMerge,how='inner',on='pfam_annotation')
pv_sa_cali =  pv_sa_cali.drop_duplicates(subset='genes_x',ignore_index=True)
common_gm_bck_cali = pd.merge(common_gm_annot,stMerge,how="inner",on="pfam_annotation")
common_gm_bck_cali =  common_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)

In [479]:
exclusive_sa_annot = exclusive_sa_annot.rename(columns={"pfam_annotation_x":"pfam_annotation"})
exclusive_sa_cali = pd.merge(exclusive_sa_annot,stMerge,how='inner',on='pfam_annotation')
exclusive_sa_cali =  exclusive_sa_cali.drop_duplicates(subset='genes_x',ignore_index=True)

In [480]:
exclusive_pv_annot = exclusive_pv_annot.rename(columns={"pfam_annotation_x":"pfam_annotation"})
exclusive_pv_cali = pd.merge(exclusive_pv_annot,stMerge,how='inner',on='pfam_annotation')
exclusive_pv_cali =  exclusive_pv_cali.drop_duplicates(subset='genes_x',ignore_index=True)

In [481]:
exclusive_gm_annot = exclusive_gm_annot.rename(columns={"pfam_annotation_x":"pfam_annotation"})
exclusive_gm_cali = pd.merge(exclusive_gm_annot,stMerge,how='inner',on='pfam_annotation')
exclusive_gm_cali =  exclusive_gm_cali.drop_duplicates(subset='genes_x',ignore_index=True)

In [482]:
# All DEG nov2016 + cali trim

all_annot_nov2016 = pd.concat([sa_gm_cali,pv_gm_cali],ignore_index=True)
all_annot_nov2016 = all_annot_nov2016.drop_duplicates(ignore_index=False)
all_annot_nov2016

,genes_x,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code,function,protein,pfam_domains,genes_y
0,STRG.22011,8530.028698,2.474850,0.333569,7.419296,1.177441e-13,7.845681e-11,Collagen,PF01391,structural constituent of cuticle,-,"C1q,Collagen,Gal_Lectin",Spis_XP_022788415_1
1,STRG.22010,5229.475438,2.549502,0.356572,7.150039,8.675340e-13,4.335501e-10,Collagen,PF01391,structural constituent of cuticle,-,"C1q,Collagen,Gal_Lectin",Spis_XP_022788415_1
2,STRG.14774,608.868066,3.072180,0.438435,7.007153,2.432159e-12,9.922217e-10,Collagen,PF01391,-,-,-,Spis_XP_022788415_1
3,STRG.22004,937.066409,2.576957,0.432385,5.959874,2.524323e-09,3.578810e-07,Collagen,PF01391,carbohydrate binding,eva-1,"C8,Collagen,Gal_Lectin,TIL,VWD",Spis_XP_022788415_1
4,STRG.10875,8.934825,3.578124,1.093298,3.272782,1.064946e-03,2.078933e-02,Collagen,PF01391,Peroxidasin homolog (Drosophila)-like,PXDNL,"An_peroxidase,I-set,LRR_8,VWC",Spis_XP_022788415_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,STRG.12912,8.960920,3.255305,0.868939,3.746300,1.794620e-04,1.007617e-02,DOMON,PF03351,dopamine beta-monooxygenase activity,-,"Cu2_monoox_C,Cu2_monooxygen,DOMON",Spis_XP_022788453_1
179,STRG.8568,32.762798,2.091424,0.567636,3.684443,2.292035e-04,1.207940e-02,7tm_1,PF00001,Belongs to the G-protein coupled receptor 1 fa...,OPRK1,7tm_1,Spis_XP_022790491_1
180,STRG.11343,24.885520,2.363259,0.753357,3.136971,1.707028e-03,4.948607e-02,7tm_1,PF00001,Belongs to the G-protein coupled receptor 1 fa...,-,7tm_1,Spis_XP_022790491_1
181,STRG.7523,18.298278,-7.105869,2.073891,-3.426347,6.117592e-04,2.511169e-02,PX,PF00787,leptin receptor binding,SNX4,PX,Spis_XP_022780524_1


In [483]:
candidateListGenerator(A_all_annot,[exclusive_gm_cali],'nov2016_gm')

In [484]:
# DESeq2 comparison of DE genes between samples - may2018

In [485]:
rPath = os.getcwd()
candidatesGenesPath = rPath + "/../R_scripts/candidateGenes.csv"
dataPath = rPath + "/../../data/net/7_deseq2/annotatedGenome/adult/DESeq2_results_adult_trueTransplant_"

may2018_pv_pv_bck_VS_gm_gm_bck_path = dataPath+"pv_pv_bck_VS_gm_gm_bck.csv"
may2018_sp_sp_bck_VS_gm_gm_bck_path = dataPath+"sp_sp_bck_VS_gm_gm_bck.csv"
may2018_pv_pv_bck_VS_sp_sp_bck_path = dataPath+"pv_pv_bck_VS_sp_sp_bck.csv"
may2018_gm_gm_tro_VS_gm_gm_bck_path = dataPath+"gm_gm_tro_VS_gm_gm_bck.csv"
may2018_gm_pv_trt_VS_gm_gm_bck_path = dataPath+"gm_pv_trt_VS_gm_gm_bck.csv"
may2018_gm_pv_trt_VS_pv_pv_bck_path = dataPath+"gm_pv_trt_VS_pv_pv_bck.csv"
may2018_gm_sp_trt_VS_gm_gm_bck_path = dataPath+"gm_sp_trt_VS_gm_gm_bck.csv"
may2018_gm_sp_trt_VS_sp_sp_bck_path = dataPath+"gm_sp_trt_VS_sp_sp_bck.csv"
may2018_pv_gm_trt_VS_gm_gm_bck_path = dataPath+"pv_gm_trt_VS_gm_gm_bck.csv"
may2018_pv_gm_trt_VS_pv_pv_bck_path = dataPath+"pv_gm_trt_VS_pv_pv_bck.csv"
may2018_pv_pv_tro_VS_pv_pv_bck_path = dataPath+"pv_pv_tro_VS_pv_pv_bck.csv"
may2018_sp_gm_trt_VS_gm_gm_bck_path = dataPath+"sp_gm_trt_VS_gm_gm_bck.csv"
may2018_sp_gm_trt_VS_sp_sp_bck_path = dataPath+"sp_gm_trt_VS_sp_sp_bck.csv"
may2018_sp_sp_tro_VS_sp_sp_bck_path = dataPath+"sp_sp_tro_VS_sp_sp_bck.csv"
may2018_sp_sp_tro_VS_gm_gm_bck_path = dataPath+"sp_sp_tro_VS_gm_gm_bck.csv"
may2018_pv_pv_tro_VS_gm_gm_bck_path = dataPath+"pv_pv_tro_VS_gm_gm_bck.csv"

In [486]:
may2018_pv_pv_bck_VS_gm_gm_bck = pd.read_csv(may2018_pv_pv_bck_VS_gm_gm_bck_path,sep=',')
may2018_sp_sp_bck_VS_gm_gm_bck = pd.read_csv(may2018_sp_sp_bck_VS_gm_gm_bck_path,sep=',')
may2018_pv_pv_bck_VS_sp_sp_bck = pd.read_csv(may2018_pv_pv_bck_VS_sp_sp_bck_path,sep=',')
may2018_gm_gm_tro_VS_gm_gm_bck = pd.read_csv(may2018_gm_gm_tro_VS_gm_gm_bck_path,sep=',')
may2018_gm_pv_trt_VS_gm_gm_bck = pd.read_csv(may2018_gm_pv_trt_VS_gm_gm_bck_path,sep=',')
may2018_gm_pv_trt_VS_pv_pv_bck = pd.read_csv(may2018_gm_pv_trt_VS_pv_pv_bck_path,sep=',')
may2018_gm_sp_trt_VS_gm_gm_bck = pd.read_csv(may2018_gm_sp_trt_VS_gm_gm_bck_path,sep=',')
may2018_gm_sp_trt_VS_sp_sp_bck = pd.read_csv(may2018_gm_sp_trt_VS_sp_sp_bck_path,sep=',')
may2018_pv_gm_trt_VS_gm_gm_bck = pd.read_csv(may2018_pv_gm_trt_VS_gm_gm_bck_path,sep=',')
may2018_pv_gm_trt_VS_pv_pv_bck = pd.read_csv(may2018_pv_gm_trt_VS_pv_pv_bck_path,sep=',')
may2018_pv_pv_tro_VS_pv_pv_bck = pd.read_csv(may2018_pv_pv_tro_VS_pv_pv_bck_path,sep=',')
may2018_sp_gm_trt_VS_gm_gm_bck = pd.read_csv(may2018_sp_gm_trt_VS_gm_gm_bck_path,sep=',')
may2018_sp_gm_trt_VS_sp_sp_bck = pd.read_csv(may2018_sp_gm_trt_VS_sp_sp_bck_path,sep=',')
may2018_sp_sp_tro_VS_sp_sp_bck = pd.read_csv(may2018_sp_sp_tro_VS_sp_sp_bck_path,sep=',')
may2018_sp_sp_tro_VS_gm_gm_bck = pd.read_csv(may2018_sp_sp_tro_VS_gm_gm_bck_path,sep=',')
may2018_pv_pv_tro_VS_gm_gm_bck = pd.read_csv(may2018_pv_pv_tro_VS_gm_gm_bck_path,sep=',')

In [487]:
may2018_pv_pv_bck_VS_gm_gm_bck = may2018_pv_pv_bck_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
may2018_sp_sp_bck_VS_gm_gm_bck = may2018_sp_sp_bck_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
may2018_pv_pv_bck_VS_sp_sp_bck = may2018_pv_pv_bck_VS_sp_sp_bck.rename(columns={'Unnamed: 0':'genes'})
may2018_gm_gm_tro_VS_gm_gm_bck = may2018_gm_gm_tro_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
may2018_gm_pv_trt_VS_gm_gm_bck = may2018_gm_pv_trt_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
may2018_gm_pv_trt_VS_pv_pv_bck = may2018_gm_pv_trt_VS_pv_pv_bck.rename(columns={'Unnamed: 0':'genes'})
may2018_gm_sp_trt_VS_gm_gm_bck = may2018_gm_sp_trt_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
may2018_gm_sp_trt_VS_sp_sp_bck = may2018_gm_sp_trt_VS_sp_sp_bck.rename(columns={'Unnamed: 0':'genes'})
may2018_pv_gm_trt_VS_gm_gm_bck = may2018_pv_gm_trt_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
may2018_pv_gm_trt_VS_pv_pv_bck = may2018_pv_gm_trt_VS_pv_pv_bck.rename(columns={'Unnamed: 0':'genes'})
may2018_pv_pv_tro_VS_pv_pv_bck = may2018_pv_pv_tro_VS_pv_pv_bck.rename(columns={'Unnamed: 0':'genes'})
may2018_sp_gm_trt_VS_gm_gm_bck = may2018_sp_gm_trt_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
may2018_sp_gm_trt_VS_sp_sp_bck = may2018_sp_gm_trt_VS_sp_sp_bck.rename(columns={'Unnamed: 0':'genes'})
may2018_sp_sp_tro_VS_sp_sp_bck = may2018_sp_sp_tro_VS_sp_sp_bck.rename(columns={'Unnamed: 0':'genes'})
may2018_sp_sp_tro_VS_gm_gm_bck = may2018_sp_sp_tro_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
may2018_pv_pv_tro_VS_gm_gm_bck = may2018_pv_pv_tro_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})

In [488]:
may2018_pv_pv_bck_VS_gm_gm_bck = may2018_pv_pv_bck_VS_gm_gm_bck[may2018_pv_pv_bck_VS_gm_gm_bck['padj']<0.05]
may2018_pv_pv_bck_VS_gm_gm_bck = may2018_pv_pv_bck_VS_gm_gm_bck[(may2018_pv_pv_bck_VS_gm_gm_bck['log2FoldChange']<-2) | (may2018_pv_pv_bck_VS_gm_gm_bck['log2FoldChange']>2)]
may2018_sp_sp_bck_VS_gm_gm_bck = may2018_sp_sp_bck_VS_gm_gm_bck[may2018_sp_sp_bck_VS_gm_gm_bck['padj']<0.05]
may2018_sp_sp_bck_VS_gm_gm_bck = may2018_sp_sp_bck_VS_gm_gm_bck[(may2018_sp_sp_bck_VS_gm_gm_bck['log2FoldChange']<-2) | (may2018_sp_sp_bck_VS_gm_gm_bck['log2FoldChange']>2)]
may2018_pv_pv_bck_VS_sp_sp_bck = may2018_pv_pv_bck_VS_sp_sp_bck[may2018_pv_pv_bck_VS_sp_sp_bck['padj']<0.05]
may2018_pv_pv_bck_VS_sp_sp_bck = may2018_pv_pv_bck_VS_sp_sp_bck[(may2018_pv_pv_bck_VS_sp_sp_bck['log2FoldChange']<-2) | (may2018_pv_pv_bck_VS_sp_sp_bck['log2FoldChange']>2)]
may2018_gm_gm_tro_VS_gm_gm_bck = may2018_gm_gm_tro_VS_gm_gm_bck[may2018_gm_gm_tro_VS_gm_gm_bck['padj']<0.05]
may2018_gm_gm_tro_VS_gm_gm_bck = may2018_gm_gm_tro_VS_gm_gm_bck[(may2018_gm_gm_tro_VS_gm_gm_bck['log2FoldChange']<-2) | (may2018_gm_gm_tro_VS_gm_gm_bck['log2FoldChange']>2)]
may2018_gm_pv_trt_VS_gm_gm_bck = may2018_gm_pv_trt_VS_gm_gm_bck[may2018_gm_pv_trt_VS_gm_gm_bck['padj']<0.05]
may2018_gm_pv_trt_VS_gm_gm_bck = may2018_gm_pv_trt_VS_gm_gm_bck[(may2018_gm_pv_trt_VS_gm_gm_bck['log2FoldChange']<-2) | (may2018_gm_pv_trt_VS_gm_gm_bck['log2FoldChange']>2)]
may2018_gm_pv_trt_VS_pv_pv_bck = may2018_gm_pv_trt_VS_pv_pv_bck[may2018_gm_pv_trt_VS_pv_pv_bck['padj']<0.05]
may2018_gm_pv_trt_VS_pv_pv_bck = may2018_gm_pv_trt_VS_pv_pv_bck[(may2018_gm_pv_trt_VS_pv_pv_bck['log2FoldChange']<-2) | (may2018_gm_pv_trt_VS_pv_pv_bck['log2FoldChange']>2)]
may2018_gm_sp_trt_VS_gm_gm_bck = may2018_gm_sp_trt_VS_gm_gm_bck[may2018_gm_sp_trt_VS_gm_gm_bck['padj']<0.05]
may2018_gm_sp_trt_VS_gm_gm_bck = may2018_gm_sp_trt_VS_gm_gm_bck[(may2018_gm_sp_trt_VS_gm_gm_bck['log2FoldChange']<-2) | (may2018_gm_sp_trt_VS_gm_gm_bck['log2FoldChange']>2)]
may2018_gm_sp_trt_VS_sp_sp_bck = may2018_gm_sp_trt_VS_sp_sp_bck[may2018_gm_sp_trt_VS_sp_sp_bck['padj']<0.05]
may2018_gm_sp_trt_VS_sp_sp_bck = may2018_gm_sp_trt_VS_sp_sp_bck[(may2018_gm_sp_trt_VS_sp_sp_bck['log2FoldChange']<-2) | (may2018_gm_sp_trt_VS_sp_sp_bck['log2FoldChange']>2)]
may2018_pv_gm_trt_VS_gm_gm_bck = may2018_pv_gm_trt_VS_gm_gm_bck[may2018_pv_gm_trt_VS_gm_gm_bck['padj']<0.05]
may2018_pv_gm_trt_VS_gm_gm_bck = may2018_pv_gm_trt_VS_gm_gm_bck[(may2018_pv_gm_trt_VS_gm_gm_bck['log2FoldChange']<-2) | (may2018_pv_gm_trt_VS_gm_gm_bck['log2FoldChange']>2)]
may2018_pv_gm_trt_VS_pv_pv_bck = may2018_pv_gm_trt_VS_pv_pv_bck[may2018_pv_gm_trt_VS_pv_pv_bck['padj']<0.05]
may2018_pv_gm_trt_VS_pv_pv_bck = may2018_pv_gm_trt_VS_pv_pv_bck[(may2018_pv_gm_trt_VS_pv_pv_bck['log2FoldChange']<-2) | (may2018_pv_gm_trt_VS_pv_pv_bck['log2FoldChange']>2)]
may2018_pv_pv_tro_VS_pv_pv_bck = may2018_pv_pv_tro_VS_pv_pv_bck[may2018_pv_pv_tro_VS_pv_pv_bck['padj']<0.05]
may2018_pv_pv_tro_VS_pv_pv_bck = may2018_pv_pv_tro_VS_pv_pv_bck[(may2018_pv_pv_tro_VS_pv_pv_bck['log2FoldChange']<-2) | (may2018_pv_pv_tro_VS_pv_pv_bck['log2FoldChange']>2)]
may2018_sp_gm_trt_VS_gm_gm_bck = may2018_sp_gm_trt_VS_gm_gm_bck[may2018_sp_gm_trt_VS_gm_gm_bck['padj']<0.05]
may2018_sp_gm_trt_VS_gm_gm_bck = may2018_sp_gm_trt_VS_gm_gm_bck[(may2018_sp_gm_trt_VS_gm_gm_bck['log2FoldChange']<-2) | (may2018_sp_gm_trt_VS_gm_gm_bck['log2FoldChange']>2)]
may2018_sp_gm_trt_VS_sp_sp_bck = may2018_sp_gm_trt_VS_sp_sp_bck[may2018_sp_gm_trt_VS_sp_sp_bck['padj']<0.05]
may2018_sp_gm_trt_VS_sp_sp_bck = may2018_sp_gm_trt_VS_sp_sp_bck[(may2018_sp_gm_trt_VS_sp_sp_bck['log2FoldChange']<-2) | (may2018_sp_gm_trt_VS_sp_sp_bck['log2FoldChange']>2)]
may2018_sp_sp_tro_VS_sp_sp_bck = may2018_sp_sp_tro_VS_sp_sp_bck[may2018_sp_sp_tro_VS_sp_sp_bck['padj']<0.05]
may2018_sp_sp_tro_VS_sp_sp_bck = may2018_sp_sp_tro_VS_sp_sp_bck[(may2018_sp_sp_tro_VS_sp_sp_bck['log2FoldChange']<-2) | (may2018_sp_sp_tro_VS_sp_sp_bck['log2FoldChange']>2)]
may2018_sp_sp_tro_VS_gm_gm_bck = may2018_sp_sp_tro_VS_gm_gm_bck[may2018_sp_sp_tro_VS_gm_gm_bck['padj']<0.05]
may2018_sp_sp_tro_VS_gm_gm_bck = may2018_sp_sp_tro_VS_gm_gm_bck[(may2018_sp_sp_tro_VS_gm_gm_bck['log2FoldChange']<-2) | (may2018_sp_sp_tro_VS_gm_gm_bck['log2FoldChange']>2)]
may2018_pv_pv_tro_VS_gm_gm_bck = may2018_pv_pv_tro_VS_gm_gm_bck[may2018_pv_pv_tro_VS_gm_gm_bck['padj']<0.05]
may2018_pv_pv_tro_VS_gm_gm_bck = may2018_pv_pv_tro_VS_gm_gm_bck[(may2018_pv_pv_tro_VS_gm_gm_bck['log2FoldChange']<-2) | (may2018_pv_pv_tro_VS_gm_gm_bck['log2FoldChange']>2)]

In [489]:
may2018_pv_pv_bck_VS_gm_gm_bck_annot = pd.merge(may2018_pv_pv_bck_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
may2018_sp_sp_bck_VS_gm_gm_bck_annot = pd.merge(may2018_sp_sp_bck_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
may2018_pv_pv_bck_VS_sp_sp_bck_annot = pd.merge(may2018_pv_pv_bck_VS_sp_sp_bck,A_all_annot,how="inner",on="genes")
may2018_gm_gm_tro_VS_gm_gm_bck_annot = pd.merge(may2018_gm_gm_tro_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
may2018_gm_pv_trt_VS_gm_gm_bck_annot = pd.merge(may2018_gm_pv_trt_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
may2018_gm_pv_trt_VS_pv_pv_bck_annot = pd.merge(may2018_gm_pv_trt_VS_pv_pv_bck,A_all_annot,how="inner",on="genes")
may2018_gm_sp_trt_VS_gm_gm_bck_annot = pd.merge(may2018_gm_sp_trt_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
may2018_gm_sp_trt_VS_sp_sp_bck_annot = pd.merge(may2018_gm_sp_trt_VS_sp_sp_bck,A_all_annot,how="inner",on="genes")
may2018_pv_gm_trt_VS_gm_gm_bck_annot = pd.merge(may2018_pv_gm_trt_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
may2018_pv_gm_trt_VS_pv_pv_bck_annot = pd.merge(may2018_pv_gm_trt_VS_pv_pv_bck,A_all_annot,how="inner",on="genes")
may2018_pv_pv_tro_VS_pv_pv_bck_annot = pd.merge(may2018_pv_pv_tro_VS_pv_pv_bck,A_all_annot,how="inner",on="genes")
may2018_sp_gm_trt_VS_gm_gm_bck_annot = pd.merge(may2018_sp_gm_trt_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
may2018_sp_gm_trt_VS_sp_sp_bck_annot = pd.merge(may2018_sp_gm_trt_VS_sp_sp_bck,A_all_annot,how="inner",on="genes")
may2018_sp_sp_tro_VS_sp_sp_bck_annot = pd.merge(may2018_sp_sp_tro_VS_sp_sp_bck,A_all_annot,how="inner",on="genes")
may2018_sp_sp_tro_VS_gm_gm_bck_annot = pd.merge(may2018_sp_sp_tro_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
may2018_pv_pv_tro_VS_gm_gm_bck_annot = pd.merge(may2018_pv_pv_tro_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")

In [490]:
may2018_pv_pv_bck_VS_gm_gm_bck_cali = pd.merge(may2018_pv_pv_bck_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_pv_pv_bck_VS_gm_gm_bck_cali =  may2018_pv_pv_bck_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
may2018_sp_sp_bck_VS_gm_gm_bck_cali = pd.merge(may2018_sp_sp_bck_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_sp_sp_bck_VS_gm_gm_bck_cali =  may2018_sp_sp_bck_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
may2018_pv_pv_bck_VS_sp_sp_bck_cali = pd.merge(may2018_pv_pv_bck_VS_sp_sp_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_pv_pv_bck_VS_sp_sp_bck_cali =  may2018_pv_pv_bck_VS_sp_sp_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
may2018_gm_gm_tro_VS_gm_gm_bck_cali = pd.merge(may2018_gm_gm_tro_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_gm_gm_tro_VS_gm_gm_bck_cali =  may2018_gm_gm_tro_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
may2018_gm_pv_trt_VS_gm_gm_bck_cali = pd.merge(may2018_gm_pv_trt_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_gm_pv_trt_VS_gm_gm_bck_cali =  may2018_gm_pv_trt_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
may2018_gm_pv_trt_VS_pv_pv_bck_cali = pd.merge(may2018_gm_pv_trt_VS_pv_pv_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_gm_pv_trt_VS_pv_pv_bck_cali =  may2018_gm_pv_trt_VS_pv_pv_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
may2018_gm_sp_trt_VS_gm_gm_bck_cali = pd.merge(may2018_gm_sp_trt_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_gm_sp_trt_VS_gm_gm_bck_cali =  may2018_gm_sp_trt_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
may2018_gm_sp_trt_VS_sp_sp_bck_cali = pd.merge(may2018_gm_sp_trt_VS_sp_sp_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_gm_sp_trt_VS_sp_sp_bck_cali =  may2018_gm_sp_trt_VS_sp_sp_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
may2018_pv_gm_trt_VS_gm_gm_bck_cali = pd.merge(may2018_pv_gm_trt_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_pv_gm_trt_VS_gm_gm_bck_cali =  may2018_pv_gm_trt_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
may2018_pv_gm_trt_VS_pv_pv_bck_cali = pd.merge(may2018_pv_gm_trt_VS_pv_pv_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_pv_gm_trt_VS_pv_pv_bck_cali =  may2018_pv_gm_trt_VS_pv_pv_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
may2018_pv_pv_tro_VS_pv_pv_bck_cali = pd.merge(may2018_pv_pv_tro_VS_pv_pv_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_pv_pv_tro_VS_pv_pv_bck_cali =  may2018_pv_pv_tro_VS_pv_pv_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
may2018_sp_gm_trt_VS_gm_gm_bck_cali = pd.merge(may2018_sp_gm_trt_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_sp_gm_trt_VS_gm_gm_bck_cali =  may2018_sp_gm_trt_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
may2018_sp_gm_trt_VS_sp_sp_bck_cali = pd.merge(may2018_sp_gm_trt_VS_sp_sp_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_sp_gm_trt_VS_sp_sp_bck_cali =  may2018_sp_gm_trt_VS_sp_sp_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
may2018_sp_sp_tro_VS_sp_sp_bck_cali = pd.merge(may2018_sp_sp_tro_VS_sp_sp_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_sp_sp_tro_VS_sp_sp_bck_cali =  may2018_sp_sp_tro_VS_sp_sp_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
may2018_sp_sp_tro_VS_gm_gm_bck_cali = pd.merge(may2018_sp_sp_tro_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_sp_sp_tro_VS_gm_gm_bck_cali =  may2018_sp_sp_tro_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
may2018_pv_pv_tro_VS_gm_gm_bck_cali = pd.merge(may2018_pv_pv_tro_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
may2018_pv_pv_tro_VS_gm_gm_bck_cali =  may2018_pv_pv_tro_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)

In [491]:
# bck may2018 + cali trim
may2018_bck = pd.concat([may2018_pv_pv_bck_VS_gm_gm_bck_cali,may2018_sp_sp_bck_VS_gm_gm_bck_cali],ignore_index=True)
may2018_bck = may2018_bck.drop_duplicates(ignore_index=False)
may2018_bck

,genes_x,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code,function,protein,pfam_domains,genes_y
0,STRG.40375,55.712225,-23.286352,2.973896,-7.830250,4.868996e-15,7.787472e-11,cEGF,PF12662,coreceptor activity involved in Wnt signaling ...,-,"EGF_CA,MAM,TSP_1",Spis_XP_022801333_1
1,STRG.40373,93.662560,-23.892940,3.244879,-7.363275,1.794517e-13,9.567166e-10,cEGF,PF12662,sequestering of TGFbeta in extracellular matrix,-,"EGF_CA,FXa_inhibition,HYR,Sushi,cEGF,hEGF",Spis_XP_022801333_1
2,STRG.35207,24.522878,-21.920011,3.660843,-5.987696,2.128349e-09,2.761307e-06,cEGF,PF12662,sequestering of TGFbeta in extracellular matrix,-,"EGF,EGF_3,EGF_CA,NIDO,SEA,hEGF",Spis_XP_022801333_1
3,STRG.33509,120.407900,-2.386024,0.315594,-7.560423,4.017621e-14,3.212891e-10,BCCT,PF02028,"BCCT, betaine/carnitine/choline family transpo...",-,"BCCT,PLAC8",Spis_XP_022786186_1
4,STRG.2761,37.653008,20.152891,2.957588,6.813961,9.494725e-12,3.037173e-08,EGF_3,PF12947,hyaluronic acid binding,-,Lectin_C,Spis_XP_022791446_1
5,STRG.12128,71.991365,2.173151,0.437218,4.970409,6.681169e-07,4.274345e-04,I-set,PF07679,"Tyrosine kinase, catalytic domain",-,Pkinase_Tyr,Spis_XP_022810032_1
6,STRG.37804,91.171164,-3.058686,0.909471,-3.363150,7.705855e-04,3.465581e-02,I-set,PF07679,Ig-like domain on T-cell surface glycoprotein ...,-,Ig_3,Spis_XP_022810032_1
7,STRG.37806,41.688444,-3.666577,1.143346,-3.206884,1.341811e-03,4.607308e-02,I-set,PF07679,Ig-like domain on T-cell surface glycoprotein ...,-,Ig_3,Spis_XP_022810032_1
8,STRG.28834,53.577752,20.732263,4.488816,4.618649,3.862473e-06,1.625694e-03,Pkinase,PF00069,JUN kinase kinase kinase activity,MAP3K1,"Pkinase,SWIM",Spis_XP_022789892_1
9,STRG.19337,56.004382,7.433907,1.994305,3.727567,1.933369e-04,1.790910e-02,Pkinase,PF00069,Protein tyrosine kinase,-,"Kelch_1,Kelch_6,Pkinase,Pkinase_Tyr",Spis_XP_022789892_1


In [492]:
candidateListGenerator(A_all_annot,[may2018_bck],'may2018_bck')

In [493]:
# exclusive gm bck may2018 + cali trim
common_gm_may2018 = pd.merge(may2018_pv_pv_bck_VS_gm_gm_bck,may2018_sp_sp_bck_VS_gm_gm_bck,how="inner",on="genes")
common_gm_may2018_annot = pd.merge(common_gm_may2018,A_all_annot,how="inner",on="genes")
excl_may2018_gm_bck_annot = pd.merge(common_gm_may2018_annot,may2018_pv_pv_bck_VS_sp_sp_bck_annot,how='left',on='genes')
excl_may2018_gm_bck_annot = excl_may2018_gm_bck_annot.rename(columns={"pfam_annotation_x":"pfam_annotation"})
excl_may2018_gm_bck_cali = pd.merge(excl_may2018_gm_bck_annot,stMerge,how='inner',on='pfam_annotation')
excl_may2018_gm_bck_cali =  excl_may2018_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
excl_may2018_gm_bck_cali

,genes_x,baseMean_x,log2FoldChange_x,lfcSE_x,stat_x,pvalue_x,padj_x,baseMean_y,log2FoldChange_y,lfcSE_y,...,lfcSE,stat,pvalue,padj,pfam_annotation_y,pfam_code_y,function_y,protein_y,pfam_domains_y,genes_y
0,STRG.2761,37.653008,20.152891,2.957588,6.813961,9.494725e-12,3.037173e-08,37.653008,13.798174,3.053451,...,2.957588,6.813961,9.494725e-12,3.037173e-08,EGF_3,PF12947,hyaluronic acid binding,-,Lectin_C,Spis_XP_022791446_1
1,STRG.28834,53.577752,20.732263,4.488816,4.618649,3.862473e-06,1.625694e-03,53.577752,18.822958,4.490368,...,4.488816,4.618649,3.862473e-06,1.625694e-03,Pkinase,PF00069,JUN kinase kinase kinase activity,MAP3K1,"Pkinase,SWIM",Spis_XP_022789892_1
2,STRG.34588,29.095221,19.045277,4.297428,4.431785,9.345600e-06,2.989470e-03,29.095221,18.076178,4.299286,...,4.297428,4.431785,9.345600e-06,2.989470e-03,Collagen,PF01391,Collagen triple helix,-,Collagen,Spis_XP_022788415_1
3,STRG.36405,35.631807,4.305543,1.102055,3.906829,9.351515e-05,1.133092e-02,35.631807,5.543090,1.096736,...,1.102055,3.906829,9.351515e-05,1.133092e-02,HEPN_DZIP3,PF18738,NACHT domain,-,NACHT,Spis_XP_022802583_1
4,STRG.4914,63.133649,6.316317,1.663299,3.797463,1.461848e-04,1.479798e-02,63.133649,6.793733,1.662301,...,1.663299,3.797463,1.461848e-04,1.479798e-02,Astacin,PF01400,metalloendopeptidase activity,nas-15,"Astacin,ShK",Spis_XP_022785469_1
5,STRG.33979,40.703032,6.569579,1.968578,3.337221,8.462059e-04,3.667809e-02,40.703032,7.364064,1.966509,...,1.968578,3.337221,8.462059e-04,3.667809e-02,Cytochrom_B561,PF03188,Ferric-chelate reductase,FRRS1,"Cytochrom_B561,DOMON,Reeler",Spis_XP_022786172_1


In [494]:
candidateListGenerator(A_all_annot,[excl_may2018_gm_bck_cali],'may2018_excl_gm_bck')

In [495]:
# tro may2018 + cali trim
may2018_tro = pd.concat([may2018_pv_pv_tro_VS_gm_gm_bck_cali,may2018_sp_sp_tro_VS_gm_gm_bck_cali],ignore_index=True)
may2018_tro = may2018_tro.drop_duplicates(ignore_index=False)
may2018_tro

,genes_x,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code,function,protein,pfam_domains,genes_y
0,STRG.2761,37.653008,17.579531,2.631073,6.681507,2.364975e-11,6.206708e-08,EGF_3,PF12947,hyaluronic acid binding,-,Lectin_C,Spis_XP_022791446_1
1,STRG.24014,5.925274,3.053701,1.061689,2.876268,4.024083e-03,4.995029e-02,EGF_3,PF12947,Coagulation factor VIII,F8,"Cu-oxidase,Cu-oxidase_2,Cu-oxidase_3,F5_F8_type_C",Spis_XP_022791446_1
2,STRG.14634,97.080761,3.122081,0.544530,5.733530,9.836144e-09,8.213628e-06,CUB,PF00431,metalloendopeptidase activity,-,"7tm_2,CUB,GPS,Lectin_C",Spis_XP_022796523_1
3,STRG.27516,26.411497,3.195707,0.863475,3.700984,2.147654e-04,7.859474e-03,CUB,PF00431,metalloendopeptidase activity,TLL1,"Astacin,CUB,EGF_CA,FXa_inhibition",Spis_XP_022796523_1
4,STRG.14529,8.254758,3.908466,1.235653,3.163077,1.561109e-03,2.798327e-02,CUB,PF00431,"Domain first found in C1r, C1s, uEGF, and bone...",-,CUB,Spis_XP_022796523_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,STRG.20517,5.143903,6.247216,1.853078,3.371264,7.482404e-04,1.746556e-02,TSP_1,PF00090,A disintegrin and metalloproteinase with throm...,-,"Pep_M12B_propep,TSP_1",Spis_XP_022793832_1
169,STRG.20037,8.653469,-2.136727,0.638332,-3.347359,8.158534e-04,1.826543e-02,TSP_1,PF00090,negative regulation of cellular response to he...,ADAMTS9,"ADAM_spacer1,GON,Pep_M12B_propep,Reprolysin,TSP_1",Spis_XP_022793832_1
170,STRG.38087,9.877640,-2.279062,0.687776,-3.313670,9.208018e-04,1.945974e-02,Prominin,PF05478,glomerular parietal epithelial cell differenti...,-,Prominin,Spis_XP_022799086_1
171,STRG.11025,38.825622,2.391591,0.821561,2.911032,3.602375e-03,4.188613e-02,Cupin_8,PF13621,peptidyl-lysine hydroxylation to 5-hydroxy-L-l...,-,Cupin_8,Spis_XP_022783445_1


In [496]:
candidateListGenerator(A_all_annot,[may2018_tro],'may2018_tro')

In [497]:
# exclusive gm tro may2018 + cali trim
common_gm_may2018_tro = pd.merge(may2018_pv_pv_tro_VS_gm_gm_bck,may2018_sp_sp_tro_VS_gm_gm_bck,how="inner",on="genes")
common_gm_may2018_tro_annot = pd.merge(common_gm_may2018_tro,A_all_annot,how="inner",on="genes")
excl_may2018_gm_tro_annot = pd.merge(common_gm_may2018_tro_annot,may2018_pv_pv_bck_VS_sp_sp_bck_annot,how='left',on='genes')
excl_may2018_gm_tro_annot = excl_may2018_gm_tro_annot.rename(columns={"pfam_annotation_x":"pfam_annotation"})
excl_may2018_gm_tro_cali = pd.merge(excl_may2018_gm_tro_annot,stMerge,how='inner',on='pfam_annotation')
excl_may2018_gm_tro_cali =  excl_may2018_gm_tro_cali.drop_duplicates(subset='genes_x',ignore_index=True)
excl_may2018_gm_tro_cali

,genes_x,baseMean_x,log2FoldChange_x,lfcSE_x,stat_x,pvalue_x,padj_x,baseMean_y,log2FoldChange_y,lfcSE_y,...,lfcSE,stat,pvalue,padj,pfam_annotation_y,pfam_code_y,function_y,protein_y,pfam_domains_y,genes_y
0,STRG.2761,37.653008,17.579531,2.631073,6.681507,2.364975e-11,6.206708e-08,37.653008,19.658375,2.697531,...,2.957588,6.813961,9.494725e-12,3.037173e-08,EGF_3,PF12947,hyaluronic acid binding,-,Lectin_C,Spis_XP_022791446_1
1,STRG.28834,53.577752,20.316906,3.931690,5.167474,2.372786e-07,7.993400e-05,53.577752,19.898726,4.049570,...,4.488816,4.618649,3.862473e-06,1.625694e-03,Pkinase,PF00069,JUN kinase kinase kinase activity,MAP3K1,"Pkinase,SWIM",Spis_XP_022789892_1
2,STRG.19337,56.004382,7.103165,1.776551,3.998290,6.380166e-05,3.662814e-03,56.004382,6.275740,1.823147,...,1.994305,3.727567,1.933369e-04,1.790910e-02,Pkinase,PF00069,Protein tyrosine kinase,-,"Kelch_1,Kelch_6,Pkinase,Pkinase_Tyr",Spis_XP_022789892_1
3,STRG.41408,14.233177,18.093000,5.341215,3.387432,7.055028e-04,1.663773e-02,14.233177,17.663912,5.508855,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022789892_1
4,STRG.31990,13.477025,5.349403,1.657225,3.227928,1.246902e-03,2.416332e-02,13.477025,6.091518,1.682860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022789892_1
5,STRG.4914,63.133649,7.656899,1.525779,5.018353,5.211630e-07,1.387578e-04,63.133649,8.156733,1.553594,...,1.663299,3.797463,1.461848e-04,1.479798e-02,Astacin,PF01400,metalloendopeptidase activity,nas-15,"Astacin,ShK",Spis_XP_022785469_1
6,STRG.36405,35.631807,4.991692,1.002862,4.977445,6.442922e-07,1.537181e-04,35.631807,4.974398,1.023458,...,1.102055,3.906829,9.351515e-05,1.133092e-02,HEPN_DZIP3,PF18738,NACHT domain,-,NACHT,Spis_XP_022802583_1
7,STRG.2026,75.682177,3.280374,0.685726,4.783796,1.720150e-06,3.183280e-04,75.682177,2.310852,0.708707,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022788415_1
8,STRG.41621,9.333098,4.066565,1.054715,3.855607,1.154429e-04,5.117694e-03,9.333098,3.079322,1.085714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022788415_1
9,STRG.38369,57.543592,2.406293,0.691363,3.480503,5.004727e-04,1.348121e-02,57.543592,2.069848,0.714075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022788415_1


In [498]:
candidateListGenerator(A_all_annot,[excl_may2018_gm_tro_cali],'may2018_excl_gm_tro')

In [499]:
# trt may2018 + cali trim
may2018_trt = pd.concat([may2018_pv_gm_trt_VS_gm_gm_bck_cali,may2018_pv_gm_trt_VS_pv_pv_bck_cali,may2018_sp_gm_trt_VS_gm_gm_bck_cali,may2018_sp_gm_trt_VS_sp_sp_bck_cali],ignore_index=True)
may2018_trt = may2018_tro.drop_duplicates(ignore_index=False)
may2018_trt

,genes_x,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code,function,protein,pfam_domains,genes_y
0,STRG.2761,37.653008,17.579531,2.631073,6.681507,2.364975e-11,6.206708e-08,EGF_3,PF12947,hyaluronic acid binding,-,Lectin_C,Spis_XP_022791446_1
1,STRG.24014,5.925274,3.053701,1.061689,2.876268,4.024083e-03,4.995029e-02,EGF_3,PF12947,Coagulation factor VIII,F8,"Cu-oxidase,Cu-oxidase_2,Cu-oxidase_3,F5_F8_type_C",Spis_XP_022791446_1
2,STRG.14634,97.080761,3.122081,0.544530,5.733530,9.836144e-09,8.213628e-06,CUB,PF00431,metalloendopeptidase activity,-,"7tm_2,CUB,GPS,Lectin_C",Spis_XP_022796523_1
3,STRG.27516,26.411497,3.195707,0.863475,3.700984,2.147654e-04,7.859474e-03,CUB,PF00431,metalloendopeptidase activity,TLL1,"Astacin,CUB,EGF_CA,FXa_inhibition",Spis_XP_022796523_1
4,STRG.14529,8.254758,3.908466,1.235653,3.163077,1.561109e-03,2.798327e-02,CUB,PF00431,"Domain first found in C1r, C1s, uEGF, and bone...",-,CUB,Spis_XP_022796523_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,STRG.20517,5.143903,6.247216,1.853078,3.371264,7.482404e-04,1.746556e-02,TSP_1,PF00090,A disintegrin and metalloproteinase with throm...,-,"Pep_M12B_propep,TSP_1",Spis_XP_022793832_1
169,STRG.20037,8.653469,-2.136727,0.638332,-3.347359,8.158534e-04,1.826543e-02,TSP_1,PF00090,negative regulation of cellular response to he...,ADAMTS9,"ADAM_spacer1,GON,Pep_M12B_propep,Reprolysin,TSP_1",Spis_XP_022793832_1
170,STRG.38087,9.877640,-2.279062,0.687776,-3.313670,9.208018e-04,1.945974e-02,Prominin,PF05478,glomerular parietal epithelial cell differenti...,-,Prominin,Spis_XP_022799086_1
171,STRG.11025,38.825622,2.391591,0.821561,2.911032,3.602375e-03,4.188613e-02,Cupin_8,PF13621,peptidyl-lysine hydroxylation to 5-hydroxy-L-l...,-,Cupin_8,Spis_XP_022783445_1


In [500]:
candidateListGenerator(A_all_annot,[may2018_trt],'may2018_trt')

In [501]:
# exclusive gm trt may2018 + cali trim
common_gm_may2018_trt = pd.merge(may2018_pv_gm_trt_VS_gm_gm_bck,may2018_sp_gm_trt_VS_gm_gm_bck,how="inner",on="genes")
common_gm_may2018_trt_annot = pd.merge(common_gm_may2018_trt,A_all_annot,how="inner",on="genes")
excl_may2018_gm_trt_annot = pd.merge(common_gm_may2018_trt_annot,may2018_pv_gm_trt_VS_pv_pv_bck_annot,how='left',on='genes')
excl_may2018_gm_trt_annot = pd.merge(common_gm_may2018_trt_annot,may2018_sp_gm_trt_VS_sp_sp_bck_annot,how='left',on='genes')
excl_may2018_gm_trt_annot = excl_may2018_gm_trt_annot.rename(columns={"pfam_annotation_x":"pfam_annotation"})
excl_may2018_gm_trt_cali = pd.merge(excl_may2018_gm_trt_annot,stMerge,how='inner',on='pfam_annotation')
excl_may2018_gm_trt_cali =  excl_may2018_gm_trt_cali.drop_duplicates(subset='genes_x',ignore_index=True)
excl_may2018_gm_trt_cali

,genes_x,baseMean_x,log2FoldChange_x,lfcSE_x,stat_x,pvalue_x,padj_x,baseMean_y,log2FoldChange_y,lfcSE_y,...,lfcSE,stat,pvalue,padj,pfam_annotation_y,pfam_code_y,function_y,protein_y,pfam_domains_y,genes_y
0,STRG.2761,37.653008,17.476088,2.701745,6.468444,9.901739e-11,3.423856e-07,37.653008,18.307121,2.799811,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022791446_1
1,STRG.12145,39.539459,4.719286,0.992612,4.754411,1.990255e-06,2.428931e-03,39.539459,4.218269,1.027592,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022783415_1
2,STRG.34588,29.095221,18.058768,3.880698,4.653485,3.263716e-06,3.385616e-03,29.095221,16.926846,4.045634,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022788415_1
3,STRG.28834,53.577752,17.324481,4.053060,4.274421,1.916349e-05,1.370983e-02,53.577752,19.064641,4.220276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022789892_1
4,STRG.36405,35.631807,4.227671,1.026119,4.120061,3.787728e-05,2.381333e-02,35.631807,4.578267,1.052983,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022802583_1


In [502]:
candidateListGenerator(A_all_annot,[excl_may2018_gm_trt_cali],'may2018_excl_gm_trt')

In [503]:
# DESeq2 comparison of DE genes between samples - September2018

In [504]:
rPath = os.getcwd()

dataPath = rPath + "/../../data/net/7_deseq2/annotatedGenome/adult/DESeq2_results_adult_gardenShort_"

sept2018_pv_pv_bck_VS_gm_gm_bck_path = dataPath+"pv_pv_bck_VS_gm_gm_bck.csv"
sept2018_sp_sp_bck_VS_gm_gm_bck_path = dataPath+"sp_sp_bck_VS_gm_gm_bck.csv"
sept2018_pv_pv_bck_VS_sp_sp_bck_path = dataPath+"pv_pv_bck_VS_sp_sp_bck.csv"
sept2018_gm_gm_gas_VS_gm_gm_bck_path = dataPath+"gm_gm_gas_VS_gm_gm_bck.csv"
sept2018_gm_pv_gas_VS_gm_gm_bck_path = dataPath+"gm_pv_gas_VS_gm_gm_bck.csv"
sept2018_gm_pv_gas_VS_pv_pv_bck_path = dataPath+"gm_pv_gas_VS_pv_pv_bck.csv"
sept2018_gm_sp_gas_VS_gm_gm_bck_path = dataPath+"gm_sp_gas_VS_gm_gm_bck.csv"
sept2018_gm_sp_gas_VS_sp_sp_bck_path = dataPath+"gm_sp_gas_VS_sp_sp_bck.csv"
sept2018_pv_gm_gas_VS_gm_gm_bck_path = dataPath+"pv_gm_gas_VS_gm_gm_bck.csv"
sept2018_pv_gm_gas_VS_pv_pv_bck_path = dataPath+"pv_gm_gas_VS_pv_pv_bck.csv"
sept2018_pv_pv_gas_VS_pv_pv_bck_path = dataPath+"pv_pv_gas_VS_pv_pv_bck.csv"
sept2018_sp_gm_gas_VS_gm_gm_bck_path = dataPath+"sp_gm_gas_VS_gm_gm_bck.csv"
sept2018_sp_gm_gas_VS_sp_sp_bck_path = dataPath+"sp_gm_gas_VS_sp_sp_bck.csv"
sept2018_sp_sp_gas_VS_sp_sp_bck_path = dataPath+"sp_sp_gas_VS_sp_sp_bck.csv"
sept2018_sp_sp_gas_VS_gm_gm_bck_path = dataPath+"sp_sp_gas_VS_gm_gm_bck.csv"
sept2018_pv_pv_gas_VS_gm_gm_bck_path = dataPath+"pv_pv_gas_VS_gm_gm_bck.csv"

In [505]:
sept2018_pv_pv_bck_VS_gm_gm_bck = pd.read_csv(sept2018_pv_pv_bck_VS_gm_gm_bck_path,sep=',')
sept2018_sp_sp_bck_VS_gm_gm_bck = pd.read_csv(sept2018_sp_sp_bck_VS_gm_gm_bck_path,sep=',')
sept2018_pv_pv_bck_VS_sp_sp_bck = pd.read_csv(sept2018_pv_pv_bck_VS_sp_sp_bck_path,sep=',')
sept2018_gm_gm_gas_VS_gm_gm_bck = pd.read_csv(sept2018_gm_gm_gas_VS_gm_gm_bck_path,sep=',')
sept2018_gm_pv_gas_VS_gm_gm_bck = pd.read_csv(sept2018_gm_pv_gas_VS_gm_gm_bck_path,sep=',')
sept2018_gm_pv_gas_VS_pv_pv_bck = pd.read_csv(sept2018_gm_pv_gas_VS_pv_pv_bck_path,sep=',')
sept2018_gm_sp_gas_VS_gm_gm_bck = pd.read_csv(sept2018_gm_sp_gas_VS_gm_gm_bck_path,sep=',')
sept2018_gm_sp_gas_VS_sp_sp_bck = pd.read_csv(sept2018_gm_sp_gas_VS_sp_sp_bck_path,sep=',')
sept2018_pv_gm_gas_VS_gm_gm_bck = pd.read_csv(sept2018_pv_gm_gas_VS_gm_gm_bck_path,sep=',')
sept2018_pv_gm_gas_VS_pv_pv_bck = pd.read_csv(sept2018_pv_gm_gas_VS_pv_pv_bck_path,sep=',')
sept2018_pv_pv_gas_VS_pv_pv_bck = pd.read_csv(sept2018_pv_pv_gas_VS_pv_pv_bck_path,sep=',')
sept2018_sp_gm_gas_VS_gm_gm_bck = pd.read_csv(sept2018_sp_gm_gas_VS_gm_gm_bck_path,sep=',')
sept2018_sp_gm_gas_VS_sp_sp_bck = pd.read_csv(sept2018_sp_gm_gas_VS_sp_sp_bck_path,sep=',')
sept2018_sp_sp_gas_VS_sp_sp_bck = pd.read_csv(sept2018_sp_sp_gas_VS_sp_sp_bck_path,sep=',')
sept2018_sp_sp_gas_VS_gm_gm_bck = pd.read_csv(sept2018_sp_sp_gas_VS_gm_gm_bck_path,sep=',')
sept2018_pv_pv_gas_VS_gm_gm_bck = pd.read_csv(sept2018_pv_pv_gas_VS_gm_gm_bck_path,sep=',')

In [506]:
sept2018_pv_pv_bck_VS_gm_gm_bck = sept2018_pv_pv_bck_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
sept2018_sp_sp_bck_VS_gm_gm_bck = sept2018_sp_sp_bck_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
sept2018_pv_pv_bck_VS_sp_sp_bck = sept2018_pv_pv_bck_VS_sp_sp_bck.rename(columns={'Unnamed: 0':'genes'})
sept2018_gm_gm_gas_VS_gm_gm_bck = sept2018_gm_gm_gas_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
sept2018_gm_pv_gas_VS_gm_gm_bck = sept2018_gm_pv_gas_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
sept2018_gm_pv_gas_VS_pv_pv_bck = sept2018_gm_pv_gas_VS_pv_pv_bck.rename(columns={'Unnamed: 0':'genes'})
sept2018_gm_sp_gas_VS_gm_gm_bck = sept2018_gm_sp_gas_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
sept2018_gm_sp_gas_VS_sp_sp_bck = sept2018_gm_sp_gas_VS_sp_sp_bck.rename(columns={'Unnamed: 0':'genes'})
sept2018_pv_gm_gas_VS_gm_gm_bck = sept2018_pv_gm_gas_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
sept2018_pv_gm_gas_VS_pv_pv_bck = sept2018_pv_gm_gas_VS_pv_pv_bck.rename(columns={'Unnamed: 0':'genes'})
sept2018_pv_pv_gas_VS_pv_pv_bck = sept2018_pv_pv_gas_VS_pv_pv_bck.rename(columns={'Unnamed: 0':'genes'})
sept2018_sp_gm_gas_VS_gm_gm_bck = sept2018_sp_gm_gas_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
sept2018_sp_gm_gas_VS_sp_sp_bck = sept2018_sp_gm_gas_VS_sp_sp_bck.rename(columns={'Unnamed: 0':'genes'})
sept2018_sp_sp_gas_VS_sp_sp_bck = sept2018_sp_sp_gas_VS_sp_sp_bck.rename(columns={'Unnamed: 0':'genes'})
sept2018_sp_sp_gas_VS_gm_gm_bck = sept2018_sp_sp_gas_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})
sept2018_pv_pv_gas_VS_gm_gm_bck = sept2018_pv_pv_gas_VS_gm_gm_bck.rename(columns={'Unnamed: 0':'genes'})

In [507]:
sept2018_pv_pv_bck_VS_gm_gm_bck = sept2018_pv_pv_bck_VS_gm_gm_bck[sept2018_pv_pv_bck_VS_gm_gm_bck['padj']<0.05]
sept2018_pv_pv_bck_VS_gm_gm_bck = sept2018_pv_pv_bck_VS_gm_gm_bck[(sept2018_pv_pv_bck_VS_gm_gm_bck['log2FoldChange']<-2) | (sept2018_pv_pv_bck_VS_gm_gm_bck['log2FoldChange']>2)]
sept2018_sp_sp_bck_VS_gm_gm_bck = sept2018_sp_sp_bck_VS_gm_gm_bck[sept2018_sp_sp_bck_VS_gm_gm_bck['padj']<0.05]
sept2018_sp_sp_bck_VS_gm_gm_bck = sept2018_sp_sp_bck_VS_gm_gm_bck[(sept2018_sp_sp_bck_VS_gm_gm_bck['log2FoldChange']<-2) | (sept2018_sp_sp_bck_VS_gm_gm_bck['log2FoldChange']>2)]
sept2018_pv_pv_bck_VS_sp_sp_bck = sept2018_pv_pv_bck_VS_sp_sp_bck[sept2018_pv_pv_bck_VS_sp_sp_bck['padj']<0.05]
sept2018_pv_pv_bck_VS_sp_sp_bck = sept2018_pv_pv_bck_VS_sp_sp_bck[(sept2018_pv_pv_bck_VS_sp_sp_bck['log2FoldChange']<-2) | (sept2018_pv_pv_bck_VS_sp_sp_bck['log2FoldChange']>2)]
sept2018_gm_gm_gas_VS_gm_gm_bck = sept2018_gm_gm_gas_VS_gm_gm_bck[sept2018_gm_gm_gas_VS_gm_gm_bck['padj']<0.05]
sept2018_gm_gm_gas_VS_gm_gm_bck = sept2018_gm_gm_gas_VS_gm_gm_bck[(sept2018_gm_gm_gas_VS_gm_gm_bck['log2FoldChange']<-2) | (sept2018_gm_gm_gas_VS_gm_gm_bck['log2FoldChange']>2)]
sept2018_gm_pv_gas_VS_gm_gm_bck = sept2018_gm_pv_gas_VS_gm_gm_bck[sept2018_gm_pv_gas_VS_gm_gm_bck['padj']<0.05]
sept2018_gm_pv_gas_VS_gm_gm_bck = sept2018_gm_pv_gas_VS_gm_gm_bck[(sept2018_gm_pv_gas_VS_gm_gm_bck['log2FoldChange']<-2) | (sept2018_gm_pv_gas_VS_gm_gm_bck['log2FoldChange']>2)]
sept2018_gm_pv_gas_VS_pv_pv_bck = sept2018_gm_pv_gas_VS_pv_pv_bck[sept2018_gm_pv_gas_VS_pv_pv_bck['padj']<0.05]
sept2018_gm_pv_gas_VS_pv_pv_bck = sept2018_gm_pv_gas_VS_pv_pv_bck[(sept2018_gm_pv_gas_VS_pv_pv_bck['log2FoldChange']<-2) | (sept2018_gm_pv_gas_VS_pv_pv_bck['log2FoldChange']>2)]
sept2018_gm_sp_gas_VS_gm_gm_bck = sept2018_gm_sp_gas_VS_gm_gm_bck[sept2018_gm_sp_gas_VS_gm_gm_bck['padj']<0.05]
sept2018_gm_sp_gas_VS_gm_gm_bck = sept2018_gm_sp_gas_VS_gm_gm_bck[(sept2018_gm_sp_gas_VS_gm_gm_bck['log2FoldChange']<-2) | (sept2018_gm_sp_gas_VS_gm_gm_bck['log2FoldChange']>2)]
sept2018_gm_sp_gas_VS_sp_sp_bck = sept2018_gm_sp_gas_VS_sp_sp_bck[sept2018_gm_sp_gas_VS_sp_sp_bck['padj']<0.05]
sept2018_gm_sp_gas_VS_sp_sp_bck = sept2018_gm_sp_gas_VS_sp_sp_bck[(sept2018_gm_sp_gas_VS_sp_sp_bck['log2FoldChange']<-2) | (sept2018_gm_sp_gas_VS_sp_sp_bck['log2FoldChange']>2)]
sept2018_pv_gm_gas_VS_gm_gm_bck = sept2018_pv_gm_gas_VS_gm_gm_bck[sept2018_pv_gm_gas_VS_gm_gm_bck['padj']<0.05]
sept2018_pv_gm_gas_VS_gm_gm_bck = sept2018_pv_gm_gas_VS_gm_gm_bck[(sept2018_pv_gm_gas_VS_gm_gm_bck['log2FoldChange']<-2) | (sept2018_pv_gm_gas_VS_gm_gm_bck['log2FoldChange']>2)]
sept2018_pv_gm_gas_VS_pv_pv_bck = sept2018_pv_gm_gas_VS_pv_pv_bck[sept2018_pv_gm_gas_VS_pv_pv_bck['padj']<0.05]
sept2018_pv_gm_gas_VS_pv_pv_bck = sept2018_pv_gm_gas_VS_pv_pv_bck[(sept2018_pv_gm_gas_VS_pv_pv_bck['log2FoldChange']<-2) | (sept2018_pv_gm_gas_VS_pv_pv_bck['log2FoldChange']>2)]
sept2018_pv_pv_gas_VS_pv_pv_bck = sept2018_pv_pv_gas_VS_pv_pv_bck[sept2018_pv_pv_gas_VS_pv_pv_bck['padj']<0.05]
sept2018_pv_pv_gas_VS_pv_pv_bck = sept2018_pv_pv_gas_VS_pv_pv_bck[(sept2018_pv_pv_gas_VS_pv_pv_bck['log2FoldChange']<-2) | (sept2018_pv_pv_gas_VS_pv_pv_bck['log2FoldChange']>2)]
sept2018_sp_gm_gas_VS_gm_gm_bck = sept2018_sp_gm_gas_VS_gm_gm_bck[sept2018_sp_gm_gas_VS_gm_gm_bck['padj']<0.05]
sept2018_sp_gm_gas_VS_gm_gm_bck = sept2018_sp_gm_gas_VS_gm_gm_bck[(sept2018_sp_gm_gas_VS_gm_gm_bck['log2FoldChange']<-2) | (sept2018_sp_gm_gas_VS_gm_gm_bck['log2FoldChange']>2)]
sept2018_sp_gm_gas_VS_sp_sp_bck = sept2018_sp_gm_gas_VS_sp_sp_bck[sept2018_sp_gm_gas_VS_sp_sp_bck['padj']<0.05]
sept2018_sp_gm_gas_VS_sp_sp_bck = sept2018_sp_gm_gas_VS_sp_sp_bck[(sept2018_sp_gm_gas_VS_sp_sp_bck['log2FoldChange']<-2) | (sept2018_sp_gm_gas_VS_sp_sp_bck['log2FoldChange']>2)]
sept2018_sp_sp_gas_VS_sp_sp_bck = sept2018_sp_sp_gas_VS_sp_sp_bck[sept2018_sp_sp_gas_VS_sp_sp_bck['padj']<0.05]
sept2018_sp_sp_gas_VS_sp_sp_bck = sept2018_sp_sp_gas_VS_sp_sp_bck[(sept2018_sp_sp_gas_VS_sp_sp_bck['log2FoldChange']<-2) | (sept2018_sp_sp_gas_VS_sp_sp_bck['log2FoldChange']>2)]
sept2018_sp_sp_gas_VS_gm_gm_bck = sept2018_sp_sp_gas_VS_gm_gm_bck[sept2018_sp_sp_gas_VS_gm_gm_bck['padj']<0.05]
sept2018_sp_sp_gas_VS_gm_gm_bck = sept2018_sp_sp_gas_VS_gm_gm_bck[(sept2018_sp_sp_gas_VS_gm_gm_bck['log2FoldChange']<-2) | (sept2018_sp_sp_gas_VS_gm_gm_bck['log2FoldChange']>2)]
sept2018_pv_pv_gas_VS_gm_gm_bck = sept2018_pv_pv_gas_VS_gm_gm_bck[sept2018_pv_pv_gas_VS_gm_gm_bck['padj']<0.05]
sept2018_pv_pv_gas_VS_gm_gm_bck = sept2018_pv_pv_gas_VS_gm_gm_bck[(sept2018_pv_pv_gas_VS_gm_gm_bck['log2FoldChange']<-2) | (sept2018_pv_pv_gas_VS_gm_gm_bck['log2FoldChange']>2)]

In [508]:
sept2018_pv_pv_bck_VS_gm_gm_bck_annot = pd.merge(sept2018_pv_pv_bck_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
sept2018_sp_sp_bck_VS_gm_gm_bck_annot = pd.merge(sept2018_sp_sp_bck_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
sept2018_pv_pv_bck_VS_sp_sp_bck_annot = pd.merge(sept2018_pv_pv_bck_VS_sp_sp_bck,A_all_annot,how="inner",on="genes")
sept2018_gm_gm_gas_VS_gm_gm_bck_annot = pd.merge(sept2018_gm_gm_gas_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
sept2018_gm_pv_gas_VS_gm_gm_bck_annot = pd.merge(sept2018_gm_pv_gas_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
sept2018_gm_pv_gas_VS_pv_pv_bck_annot = pd.merge(sept2018_gm_pv_gas_VS_pv_pv_bck,A_all_annot,how="inner",on="genes")
sept2018_gm_sp_gas_VS_gm_gm_bck_annot = pd.merge(sept2018_gm_sp_gas_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
sept2018_gm_sp_gas_VS_sp_sp_bck_annot = pd.merge(sept2018_gm_sp_gas_VS_sp_sp_bck,A_all_annot,how="inner",on="genes")
sept2018_pv_gm_gas_VS_gm_gm_bck_annot = pd.merge(sept2018_pv_gm_gas_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
sept2018_pv_gm_gas_VS_pv_pv_bck_annot = pd.merge(sept2018_pv_gm_gas_VS_pv_pv_bck,A_all_annot,how="inner",on="genes")
sept2018_pv_pv_gas_VS_pv_pv_bck_annot = pd.merge(sept2018_pv_pv_gas_VS_pv_pv_bck,A_all_annot,how="inner",on="genes")
sept2018_sp_gm_gas_VS_gm_gm_bck_annot = pd.merge(sept2018_sp_gm_gas_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
sept2018_sp_gm_gas_VS_sp_sp_bck_annot = pd.merge(sept2018_sp_gm_gas_VS_sp_sp_bck,A_all_annot,how="inner",on="genes")
sept2018_sp_sp_gas_VS_sp_sp_bck_annot = pd.merge(sept2018_sp_sp_gas_VS_sp_sp_bck,A_all_annot,how="inner",on="genes")
sept2018_sp_sp_gas_VS_gm_gm_bck_annot = pd.merge(sept2018_sp_sp_gas_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")
sept2018_pv_pv_gas_VS_gm_gm_bck_annot = pd.merge(sept2018_pv_pv_gas_VS_gm_gm_bck,A_all_annot,how="inner",on="genes")

In [509]:
sept2018_pv_pv_bck_VS_gm_gm_bck_cali = pd.merge(sept2018_pv_pv_bck_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_pv_pv_bck_VS_gm_gm_bck_cali =  sept2018_pv_pv_bck_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
sept2018_sp_sp_bck_VS_gm_gm_bck_cali = pd.merge(sept2018_sp_sp_bck_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_sp_sp_bck_VS_gm_gm_bck_cali =  sept2018_sp_sp_bck_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
sept2018_pv_pv_bck_VS_sp_sp_bck_cali = pd.merge(sept2018_pv_pv_bck_VS_sp_sp_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_pv_pv_bask_VS_sp_sp_bck_cali =  sept2018_pv_pv_bck_VS_sp_sp_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
sept2018_gm_gm_gas_VS_gm_gm_bck_cali = pd.merge(sept2018_gm_gm_gas_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_gm_gm_gas_VS_gm_gm_bck_cali =  sept2018_gm_gm_gas_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
sept2018_gm_pv_gas_VS_gm_gm_bck_cali = pd.merge(sept2018_gm_pv_gas_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_gm_pv_gas_VS_gm_gm_bck_cali =  sept2018_gm_pv_gas_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
sept2018_gm_pv_gas_VS_pv_pv_bck_cali = pd.merge(sept2018_gm_pv_gas_VS_pv_pv_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_gm_pv_gas_VS_pv_pv_bck_cali =  sept2018_gm_pv_gas_VS_pv_pv_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
sept2018_gm_sp_gas_VS_gm_gm_bck_cali = pd.merge(sept2018_gm_sp_gas_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_gm_sp_gas_VS_gm_gm_bck_cali =  sept2018_gm_sp_gas_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
sept2018_gm_sp_gas_VS_sp_sp_bck_cali = pd.merge(sept2018_gm_sp_gas_VS_sp_sp_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_gm_sp_gas_VS_sp_sp_bck_cali =  sept2018_gm_sp_gas_VS_sp_sp_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
sept2018_pv_gm_gas_VS_gm_gm_bck_cali = pd.merge(sept2018_pv_gm_gas_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_pv_gm_gas_VS_gm_gm_bck_cali =  sept2018_pv_gm_gas_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
sept2018_pv_gm_gas_VS_pv_pv_bck_cali = pd.merge(sept2018_pv_gm_gas_VS_pv_pv_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_pv_gm_gas_VS_pv_pv_bck_cali =  sept2018_pv_gm_gas_VS_pv_pv_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
sept2018_pv_pv_gas_VS_pv_pv_bck_cali = pd.merge(sept2018_pv_pv_gas_VS_pv_pv_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_pv_pv_gas_VS_pv_pv_bck_cali =  sept2018_pv_pv_gas_VS_pv_pv_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
sept2018_sp_gm_gas_VS_gm_gm_bck_cali = pd.merge(sept2018_sp_gm_gas_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_sp_gm_gas_VS_gm_gm_bck_cali =  sept2018_sp_gm_gas_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
sept2018_sp_gm_gas_VS_sp_sp_bck_cali = pd.merge(sept2018_sp_gm_gas_VS_sp_sp_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_sp_gm_gas_VS_sp_sp_bck_cali =  sept2018_sp_gm_gas_VS_sp_sp_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
sept2018_sp_sp_gas_VS_sp_sp_bck_cali = pd.merge(sept2018_sp_sp_gas_VS_sp_sp_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_sp_sp_gas_VS_sp_sp_bck_cali =  sept2018_sp_sp_gas_VS_sp_sp_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
sept2018_sp_sp_gas_VS_gm_gm_bck_cali = pd.merge(sept2018_sp_sp_gas_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_sp_sp_gas_VS_gm_gm_bck_cali =  sept2018_sp_sp_gas_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
sept2018_pv_pv_gas_VS_gm_gm_bck_cali = pd.merge(sept2018_pv_pv_gas_VS_gm_gm_bck_annot,stMerge,how="inner",on="pfam_annotation")
sept2018_pv_pv_gas_VS_gm_gm_bck_cali =  sept2018_pv_pv_gas_VS_gm_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)

In [510]:
# bck sept2018 + cali trim
sept2018_bck = pd.concat([sept2018_pv_pv_bck_VS_gm_gm_bck_cali,sept2018_sp_sp_bck_VS_gm_gm_bck_cali],ignore_index=True)
sept2018_bck = sept2018_bck.drop_duplicates(ignore_index=False)
sept2018_bck

,genes_x,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code,function,protein,pfam_domains,genes_y
0,STRG.40373,44.942484,-11.795119,2.750171,-4.288867,0.000018,0.008338,cEGF,PF12662,sequestering of TGFbeta in extracellular matrix,-,"EGF_CA,FXa_inhibition,HYR,Sushi,cEGF,hEGF",Spis_XP_022801333_1
1,STRG.3433,447.378525,3.769387,0.949175,3.971225,0.000072,0.019509,cEGF,PF12662,thick ascending limb development,GP2,"EGF_3,EGF_CA,Zona_pellucida",Spis_XP_022801333_1
2,STRG.4914,46.858853,5.596001,1.330648,4.205471,0.000026,0.010856,Astacin,PF01400,metalloendopeptidase activity,nas-15,"Astacin,ShK",Spis_XP_022785469_1
3,STRG.27801,208.234472,4.535779,1.087314,4.171546,0.000030,0.011705,Astacin,PF01400,Putative peptidase (DUF1758),-,"DUF1758,DUF1759,Peptidase_A17",Spis_XP_022785469_1
4,STRG.16259,281.522498,-2.606175,0.669207,-3.894424,0.000098,0.021155,Astacin,PF01400,metalloendopeptidase activity,BMP1,"Astacin,CUB,EGF_CA,FXa_inhibition",Spis_XP_022785469_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,STRG.2125,16.428098,-2.864758,0.872257,-3.284305,0.001022,0.041165,7tm_1,PF00001,7 transmembrane receptor (rhodopsin family),-,7tm_1,Spis_XP_022790491_1
64,STRG.34637,5.459687,4.329469,1.341690,3.226877,0.001251,0.046363,7tm_1,PF00001,Serpentine type 7TM GPCR chemoreceptor Srw,-,7tm_1,Spis_XP_022790491_1
65,STRG.2625,608.127765,-2.044250,0.585475,-3.491611,0.000480,0.025615,EGF_3,PF12947,fibroblast growth factor-activated receptor ac...,-,Pkinase_Tyr,Spis_XP_022791446_1
66,STRG.26118,34.270476,-7.712730,2.348154,-3.284592,0.001021,0.041165,TSP_1,PF00090,GON domain,-,GON,Spis_XP_022793832_1


In [511]:
candidateListGenerator(A_all_annot,[sept2018_bck],'sept2018_bck')

In [512]:
# exclusive gm bck sept2018 + cali trim
common_gm_sept2018 = pd.merge(sept2018_pv_pv_bck_VS_gm_gm_bck,sept2018_sp_sp_bck_VS_gm_gm_bck,how="inner",on="genes")
common_gm_sept2018_annot = pd.merge(common_gm_sept2018,A_all_annot,how="inner",on="genes")
excl_sept2018_gm_bck_annot = pd.merge(common_gm_sept2018_annot,sept2018_pv_pv_bck_VS_sp_sp_bck_annot,how='left',on='genes')
excl_sept2018_gm_bck_annot = excl_sept2018_gm_bck_annot.rename(columns={"pfam_annotation_x":"pfam_annotation"})
excl_sept2018_gm_bck_cali = pd.merge(excl_sept2018_gm_bck_annot,stMerge,how='inner',on='pfam_annotation')
excl_sept2018_gm_bck_cali =  excl_sept2018_gm_bck_cali.drop_duplicates(subset='genes_x',ignore_index=True)
excl_sept2018_gm_bck_cali

,genes_x,baseMean_x,log2FoldChange_x,lfcSE_x,stat_x,pvalue_x,padj_x,baseMean_y,log2FoldChange_y,lfcSE_y,...,lfcSE,stat,pvalue,padj,pfam_annotation_y,pfam_code_y,function_y,protein_y,pfam_domains_y,genes_y
0,STRG.40373,44.942484,-11.795119,2.750171,-4.288867,0.000018,0.008338,44.942484,-11.800054,2.750171,...,2.750171,-4.288867,0.000018,0.008338,cEGF,PF12662,sequestering of TGFbeta in extracellular matrix,-,"EGF_CA,FXa_inhibition,HYR,Sushi,cEGF,hEGF",Spis_XP_022801333_1
1,STRG.14308,230.964541,-6.818000,1.728938,-3.943462,0.000080,0.020168,230.964541,-8.969092,1.778016,...,1.728938,-3.943462,0.000080,0.020168,Lectin_C,PF00059,"C-type lectin domain family 4, member",CLEC4A,Lectin_C,Spis_XP_022809492_1
2,STRG.29449,59.617614,-3.493614,0.895059,-3.903222,0.000095,0.020844,59.617614,-5.097899,0.916834,...,0.895059,-3.903222,0.000095,0.020844,F5_F8_type_C,PF00754,Gamma-glutamyltranspeptidase,-,G_glu_transpept,Spis_XP_022783415_1
3,STRG.16259,281.522498,-2.606175,0.669207,-3.894424,0.000098,0.021155,281.522498,-2.872472,0.669663,...,0.669207,-3.894424,0.000098,0.021155,Astacin,PF01400,metalloendopeptidase activity,BMP1,"Astacin,CUB,EGF_CA,FXa_inhibition",Spis_XP_022785469_1
4,STRG.31990,15.186876,7.365152,1.964653,3.748831,0.000178,0.029763,15.186876,6.407091,1.969963,...,1.964653,3.748831,0.000178,0.029763,Pkinase,PF00069,Protein tyrosine kinase,-,"Kelch_1,Kelch_6,Pkinase,Pkinase_Tyr",Spis_XP_022789892_1
5,STRG.1170,84.795824,3.438963,0.937580,3.667914,0.000245,0.033945,84.795824,3.920355,0.936323,...,0.937580,3.667914,0.000245,0.033945,CUB,PF00431,Sidekick cell adhesion molecule,PTPRQ,"Ig_3,Y_phosphatase,fn3",Spis_XP_022796523_1
6,STRG.205,449.364909,-3.861375,1.056310,-3.655531,0.000257,0.034468,449.364909,-3.940472,1.056479,...,1.056310,-3.655531,0.000257,0.034468,Kunitz_BPTI,PF00014,serine-type endopeptidase inhibitor activity,tfpi2,"Kunitz_BPTI,Thyroglobulin_1",Spis_XP_022789848_1
7,STRG.39645,12.824407,-17.754405,4.924650,-3.605211,0.000312,0.039181,12.824407,-17.780085,4.924650,...,4.924650,-3.605211,0.000312,0.039181,EGF,PF00008,calcium ion binding,-,"EGF,EGF_CA,Ephrin_rec_like,HYR,Sushi,VWA,hEGF",Spis_XP_022805384_1
8,STRG.23793,67.501747,2.936538,0.829027,3.542151,0.000397,0.043873,67.501747,3.368432,0.827397,...,0.829027,3.542151,0.000397,0.043873,EGF,PF00008,calcium ion binding,-,"Kazal_2,Laminin_EGF,Laminin_G_2,Ldl_recept_a",Spis_XP_022805384_1
9,STRG.41361,149.909048,-6.270829,1.752739,-3.577730,0.000347,0.040178,149.909048,-7.149629,1.763375,...,1.752739,-3.577730,0.000347,0.040178,I-set,PF07679,negative regulation of negative chemotaxis,IGDCC3,"I-set,Ig_2,Ig_3,V-set,fn3,ig",Spis_XP_022810032_1


In [513]:
candidateListGenerator(A_all_annot,[excl_sept2018_gm_bck_cali],'sept2018_excl_gm_bck')

In [514]:
# gas_same sept2018 + cali trim
sept2018_gas_same = pd.concat([sept2018_pv_pv_gas_VS_gm_gm_bck_cali,sept2018_sp_sp_gas_VS_gm_gm_bck_cali],ignore_index=True)
sept2018_gas_same = sept2018_gas_same.drop_duplicates(ignore_index=False)
sept2018_gas_same

,genes_x,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code,function,protein,pfam_domains,genes_y
0,STRG.5903,26.270197,4.142622,0.696668,5.946332,2.742174e-09,0.000003,EGF,PF00008,Laminin G domain,-,"EGF,Laminin_G_2",Spis_XP_022805384_1
1,STRG.13551,57.112190,3.067811,0.684533,4.481610,7.408216e-06,0.000700,EGF,PF00008,-,-,-,Spis_XP_022805384_1
2,STRG.23793,67.501747,3.152562,0.730727,4.314284,1.601208e-05,0.001196,EGF,PF00008,calcium ion binding,-,"Kazal_2,Laminin_EGF,Laminin_G_2,Ldl_recept_a",Spis_XP_022805384_1
3,STRG.22986,40.860744,3.474007,0.910706,3.814630,1.363872e-04,0.005053,EGF,PF00008,neurogenic locus notch homolog protein,NOTCH1,"Ank,Ank_2,Ank_5,DUF3454,EGF,EGF_CA,NOD,NODP,No...",Spis_XP_022805384_1
4,STRG.40906,14.716858,-2.612019,0.706183,-3.698785,2.166339e-04,0.006744,EGF,PF00008,Lipoxygenase,-,Lipoxygenase,Spis_XP_022805384_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,STRG.11025,70.224724,2.172454,0.667026,3.256928,1.126250e-03,0.028968,Cupin_8,PF13621,peptidyl-lysine hydroxylation to 5-hydroxy-L-l...,-,Cupin_8,Spis_XP_022783445_1
260,STRG.33626,5.623330,4.932595,1.581141,3.119642,1.810708e-03,0.038448,TSP_1,PF00090,coreceptor activity involved in Wnt signaling ...,-,TSP_1,Spis_XP_022793832_1
261,STRG.20517,11.404157,3.886451,1.251296,3.105942,1.896741e-03,0.039626,TSP_1,PF00090,A disintegrin and metalloproteinase with throm...,-,"Pep_M12B_propep,TSP_1",Spis_XP_022793832_1
262,STRG.17183,464.172172,-4.355100,1.406598,-3.096194,1.960220e-03,0.040427,Trypsin,PF00089,complement activation,-,"Sushi,Trypsin,VWA",Spis_XP_022779543_1


In [515]:
candidateListGenerator(A_all_annot,[sept2018_gas_same],'sept2018_gas_same')

In [516]:
# exclusive gm gas same sept2018 + cali trim
common_gm_sept2018_gas_same = pd.merge(sept2018_pv_pv_gas_VS_gm_gm_bck,sept2018_sp_sp_gas_VS_gm_gm_bck,how="inner",on="genes")
common_gm_sept2018_gas_same_annot = pd.merge(common_gm_sept2018_gas_same,A_all_annot,how="inner",on="genes")
excl_sept2018_gm_gas_same_annot = pd.merge(common_gm_sept2018_gas_same_annot,sept2018_pv_pv_bck_VS_sp_sp_bck_annot,how='left',on='genes')
excl_sept2018_gm_gas_same_annot = excl_sept2018_gm_gas_same_annot.rename(columns={"pfam_annotation_x":"pfam_annotation"})
excl_sept2018_gm_gas_same_cali = pd.merge(excl_sept2018_gm_gas_same_annot,stMerge,how='inner',on='pfam_annotation')
excl_sept2018_gm_gas_same_cali =  excl_sept2018_gm_gas_same_cali.drop_duplicates(subset='genes_x',ignore_index=True)
excl_sept2018_gm_gas_same_cali

,genes_x,baseMean_x,log2FoldChange_x,lfcSE_x,stat_x,pvalue_x,padj_x,baseMean_y,log2FoldChange_y,lfcSE_y,...,lfcSE,stat,pvalue,padj,pfam_annotation_y,pfam_code_y,function_y,protein_y,pfam_domains_y,genes_y
0,STRG.5903,26.270197,4.142622,0.696668,5.946332,2.742174e-09,0.000003,26.270197,2.770677,0.702973,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022805384_1
1,STRG.13551,57.112190,3.067811,0.684533,4.481610,7.408216e-06,0.000700,57.112190,2.435513,0.685662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022805384_1
2,STRG.23793,67.501747,3.152562,0.730727,4.314284,1.601208e-05,0.001196,67.501747,3.752766,0.729260,...,0.829027,3.542151,0.000397,0.043873,EGF,PF00008,calcium ion binding,-,"Kazal_2,Laminin_EGF,Laminin_G_2,Ldl_recept_a",Spis_XP_022805384_1
3,STRG.30263,7.180216,4.077092,1.178040,3.460911,5.383516e-04,0.011785,7.180216,3.799189,1.179914,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022805384_1
4,STRG.4914,46.858853,6.436839,1.193231,5.394463,6.872875e-08,0.000033,46.858853,5.405909,1.194210,...,1.330648,4.205471,0.000026,0.010856,Astacin,PF01400,metalloendopeptidase activity,nas-15,"Astacin,ShK",Spis_XP_022785469_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,STRG.31990,15.186876,6.230234,1.804030,3.453509,5.533433e-04,0.012009,15.186876,7.136580,1.799879,...,1.964653,3.748831,0.000178,0.029763,Pkinase,PF00069,Protein tyrosine kinase,-,"Kelch_1,Kelch_6,Pkinase,Pkinase_Tyr",Spis_XP_022789892_1
60,STRG.14442,24.038403,3.359716,1.021749,3.288201,1.008298e-03,0.017608,24.038403,3.472916,1.020839,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022788453_1
61,STRG.36386,9.986999,2.479440,0.805831,3.076875,2.091829e-03,0.028093,9.986999,2.812898,0.800931,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022787258_1
62,STRG.42450,3.967301,4.128880,1.397336,2.954823,3.128486e-03,0.036172,3.967301,4.250328,1.393792,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022779543_1


In [517]:
candidateListGenerator(A_all_annot,[excl_sept2018_gm_gas_same_cali],'sept2018_excl_gm_gas_same')

In [518]:
# gas sept2018 + cali trim
sept2018_gas = pd.concat([sept2018_pv_gm_gas_VS_gm_gm_bck_cali,sept2018_pv_gm_gas_VS_pv_pv_bck_cali,sept2018_sp_gm_gas_VS_gm_gm_bck_cali,sept2018_sp_gm_gas_VS_sp_sp_bck_cali],ignore_index=True)
sept2018_gas = sept2018_gas.drop_duplicates(ignore_index=False)
sept2018_gas

,genes_x,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,pfam_annotation,pfam_code,function,protein,pfam_domains,genes_y
0,STRG.5903,26.270197,3.975995,0.697040,5.704115,1.169493e-08,0.000011,EGF,PF00008,Laminin G domain,-,"EGF,Laminin_G_2",Spis_XP_022805384_1
1,STRG.13551,57.112190,3.271028,0.684102,4.781491,1.739996e-06,0.000367,EGF,PF00008,-,-,-,Spis_XP_022805384_1
2,STRG.22986,40.860744,4.041688,0.909711,4.442825,8.878521e-06,0.001069,EGF,PF00008,neurogenic locus notch homolog protein,NOTCH1,"Ank,Ank_2,Ank_5,DUF3454,EGF,EGF_CA,NOD,NODP,No...",Spis_XP_022805384_1
3,STRG.39645,12.824407,-18.171864,4.236421,-4.289438,1.791256e-05,0.001659,EGF,PF00008,calcium ion binding,-,"EGF,EGF_CA,Ephrin_rec_like,HYR,Sushi,VWA,hEGF",Spis_XP_022805384_1
4,STRG.8640,137.610808,-6.802618,1.613056,-4.217223,2.473299e-05,0.001882,EGF,PF00008,Cubilin (intrinsic factor-cobalamin receptor),CUBN,"CUB,EGF,EGF_3,EGF_CA,hEGF",Spis_XP_022805384_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,STRG.1674,6.847962,3.925200,1.351694,2.903911,3.685330e-03,0.033484,Lipase,PF00151,Belongs to the AB hydrolase superfamily. Lipas...,-,Lipase,Spis_XP_022779802_1
425,STRG.38088,4.576670,-2.411860,0.686125,-3.515190,4.394387e-04,0.008219,Prominin,PF05478,glomerular parietal epithelial cell differenti...,-,Prominin,Spis_XP_022799086_1
426,STRG.12224,195.039706,2.592851,0.747701,3.467765,5.248068e-04,0.009290,Y_phosphatase,PF00102,Receptor-type tyrosine-protein phosphatase,PTPRM,"I-set,MAM,Y_phosphatase,fn3,ig",Spis_XP_022810230_1
427,STRG.23041,134.987099,2.390872,0.737021,3.243966,1.178778e-03,0.015359,MORN,PF02493,axoneme assembly,RSPH1,MORN,Spis_XP_022778944_1


In [519]:
candidateListGenerator(A_all_annot,[sept2018_gas],'sept2018_gas_diff')

In [520]:
# exclusive gm gas sept2018 + cali trim
common_gm_sept2018_gas = pd.merge(sept2018_pv_gm_gas_VS_gm_gm_bck,sept2018_sp_gm_gas_VS_gm_gm_bck,how="inner",on="genes")
common_gm_sept2018_gas_annot = pd.merge(common_gm_sept2018_gas,A_all_annot,how="inner",on="genes")
excl_sept2018_gm_gas_diff_annot = pd.merge(common_gm_sept2018_gas_annot,sept2018_pv_gm_gas_VS_pv_pv_bck_annot,how='left',on='genes')
excl_sept2018_gm_gas_diff_annot = pd.merge(common_gm_sept2018_gas_annot,sept2018_sp_gm_gas_VS_sp_sp_bck_annot,how='left',on='genes')
excl_sept2018_gm_gas_diff_annot = excl_sept2018_gm_gas_diff_annot.rename(columns={"pfam_annotation_x":"pfam_annotation"})
excl_sept2018_gm_gas_diff_cali = pd.merge(excl_sept2018_gm_gas_diff_annot,stMerge,how='inner',on='pfam_annotation')
excl_sept2018_gm_gas_diff_cali =  excl_sept2018_gm_gas_diff_cali.drop_duplicates(subset='genes_x',ignore_index=True)
excl_sept2018_gm_gas_diff_cali

,genes_x,baseMean_x,log2FoldChange_x,lfcSE_x,stat_x,pvalue_x,padj_x,baseMean_y,log2FoldChange_y,lfcSE_y,...,lfcSE,stat,pvalue,padj,pfam_annotation_y,pfam_code_y,function_y,protein_y,pfam_domains_y,genes_y
0,STRG.5903,26.270197,3.975995,0.697040,5.704115,1.169493e-08,0.000011,26.270197,2.770677,0.702973,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022805384_1
1,STRG.13551,57.112190,3.271028,0.684102,4.781491,1.739996e-06,0.000367,57.112190,2.435513,0.685662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022805384_1
2,STRG.30263,7.180216,4.229498,1.175814,3.597081,3.218087e-04,0.009802,7.180216,3.799189,1.179914,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022805384_1
3,STRG.23793,67.501747,2.538248,0.732492,3.465224,5.297893e-04,0.013035,67.501747,3.752766,0.729260,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022805384_1
4,STRG.16167,56.206875,4.907216,0.881001,5.570045,2.546736e-08,0.000021,56.206875,4.195639,0.881745,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022798437_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,STRG.20517,11.404157,4.406341,1.248977,3.527960,4.187755e-04,0.011381,11.404157,3.886451,1.251296,...,1.520346,4.2237,0.000024,0.001311,TSP_1,PF00090,A disintegrin and metalloproteinase with throm...,-,"Pep_M12B_propep,TSP_1",Spis_XP_022793832_1
63,STRG.31990,15.186876,6.326935,1.803228,3.508672,4.503501e-04,0.011884,15.186876,7.136580,1.799879,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022789892_1
64,STRG.3783,6.757427,-3.279214,1.031823,-3.178077,1.482556e-03,0.024489,6.757427,-4.649035,1.090493,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022789892_1
65,STRG.36386,9.986999,2.602410,0.803672,3.238148,1.203083e-03,0.021663,9.986999,2.812898,0.800931,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spis_XP_022787258_1


In [521]:
candidateListGenerator(A_all_annot,[excl_sept2018_gm_gas_diff_cali],'sept2018_excl_gm_gas_diff')